## Rhetorical relations classification

1. Data cleaning: obtain the data and clean it (X, y)
 - transformations: normalization, scaling
2. Classifiers & CV-training for relations classification (cause/background/attribution/etc.)
4. Save the best classifier to ``models/label_predictor/model.pkl``
3. Feature selection
 - Find features with single unique value
 - Find collinear features
4. Classifiers & CV-training for nuclearity classification (NN/NS/SN)

Output:
 - ``models/label_predictor/*``

In [4]:
! pip install -U pandas==0.24.1
! pip install -Uq pip catboost

Requirement already up-to-date: pandas==0.24.1 in /root/.pyenv/versions/3.6.7/lib/python3.6/site-packages (0.24.1)


In [1]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from matplotlib import rcParams
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from keras.models import Model
from keras.optimizers import Adam
import tensorflow as tf
from catboost import CatBoostClassifier
from time import time
import pickle

rcParams['pdf.fonttype'] = 42
rcParams['font.sans-serif'] = 'Arial'
import seaborn as sns

sns.set_style("whitegrid")
%matplotlib inline

random_state = 42

/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.
/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version o

In [37]:
import glob
import pandas as pd

df = []
for filename in glob.glob("data/*.gold.pkl"):
    df.append(pd.read_pickle(filename))

In [38]:
df = pd.concat(df)
df = df.fillna(0.)

In [39]:
df.shape

(31491, 2591)

### 1. Data cleaning

In [40]:
TARGET = 'category_id'

df[TARGET] = df[TARGET].replace(['cause-effect_r', 'effect_r'], 'cause_r')
df[TARGET] = df[TARGET].replace(['interpretation-evaluation_r', 'conclusion_r'], 'evaluation_r')

y_stat = df[TARGET].value_counts()
drop_ys = y_stat[y_stat < 500].index

for dy in drop_ys:
    df = df[df[TARGET] != dy]
    
for dy in ['elaboration_r', 'joint_m', 'same-unit_m']:
    df = df[df[TARGET] != dy]

def find_annotation(filename):
    text = pickle.load(open(f'data/{filename}.annot.pkl', 'rb'))['text'].lower()
    for keyword in ['аннотация', 'удк', 'список литературы', 'использованной литературы']:
        if keyword in text:
            return 1.
    return 0.

df['given_annotation'] = df.filename.apply(find_annotation)

In [41]:
df.shape

(14900, 2591)

In [42]:
df[TARGET].value_counts()

cause_r          2492
contrast_m       1990
condition_r      1646
preparation_r    1625
evaluation_r     1420
sequence_m       1239
purpose_r        1204
attribution_r    1168
evidence_r        796
background_r      723
comparison_m      597
Name: category_id, dtype: int64

In [ ]:
#unique_type_list = df[TARGET].unique()
unique_type_list = ['attribution_r', 'purpose_r', 'contrast_m', 'preparation_r', 'sequence_m', 'condition_r', 'cause_r', 'evaluation_r', 'comparison_m', 'evidence_r', 'background_r']
y, X = df[TARGET].to_frame(), df.drop(TARGET, axis=1).drop(columns=['snippet_x', 'snippet_y', 'snippet_x_tmp', 'snippet_y_tmp', 'filename', 'order', 'postags_x', 'postags_y'])

In [ ]:
categorical_feature_mask = X.dtypes==object
categorical_cols = X.columns[categorical_feature_mask].tolist()

In [ ]:
categorical_cols

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

lab_encoder = LabelEncoder().fit(unique_type_list)
y = lab_encoder.transform(y)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d

class MyLabelEncoder(LabelEncoder):

    def fit(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_ = pd.Series(y)
        return self
    
lab_encoder = MyLabelEncoder().fit(unique_type_list)
y = lab_encoder.transform(y)

In [ ]:
X.shape

In [ ]:
! mkdir models/label_predictor
import pickle

pickle.dump(junk_features, open('models/label_predictor/drop_columns.pkl', 'wb'))
pickle.dump(lab_encoder, open('models/label_predictor/label_encoder.pkl', 'wb'))

In [ ]:
# X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))

# ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
# X_ohe = ohe.fit_transform(X[categorical_cols].values)
# X_ohe = pd.DataFrame(X_ohe, X.index, columns=ohe.get_feature_names(categorical_cols))

# X = X.join(
#    pd.DataFrame(X_ohe, X.index).add_prefix('cat_'), how='right'
# ).drop(columns=categorical_cols)

In [ ]:
scaler = StandardScaler().fit(X.values)

X_scaled_np = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled_np, index=X.index, columns=X.columns)

X = X_scaled_df

In [ ]:
import pickle

In [ ]:
pickle.dump(scaler, open('models/label_predictor/scaler.pkl', 'wb'))

In [ ]:
X.shape

### 2. Make different feature sets

In [ ]:
# locational = [feat for feat in list(X.keys()) if '_pair_' in feat]
# quantitative = [feat for feat in list(X.keys()) if '_count_' in feat]

# X_quant = X.drop(columns=locational)  # quantitative only 
# X_positional = X.drop(columns=quantitative)  # positional only

# lexical = ['tf_idf_y_167', 'tf_idf_y_114', 'tf_idf_y_97', 'tf_idf_y_46', 'tf_idf_x_291', 'tf_idf_x_261', 'tf_idf_x_118', 'tf_idf_x_97']
# morphosynt = ['last_X_X_y', 'first_X_X_y', 'last_X_NOUN_y', 'first_X_NOUN_y', 'first_X_y', 'first_VERB_X_y', 'first_VERB_PRON_y', 'last_VERB_NOUN_y', 'first_VERB_NOUN_y', 'first_VERB_ADV_y', 'last_VERB_ADP_y', 'first_VERB_ADP_y', 'first_VERB_ADJ_y', 'first_PRON_X_y', 'last_PRON_PRON_y', 'first_PRON_PRON_y', 'last_PRON_NOUN_y', 'first_PRON_NOUN_y', 'first_PART_X_y', 'last_PART_VERB_y', 'first_PART_VERB_y', 'first_NUM_NOUN_y', 'last_VERB_PRON_x', 'first_VERB_X_x', 'last_VERB_ADV_x', 'first_VERB_NOUN_x', 'last_VERB_NOUN_x', 'first_X_NOUN_x', 'last_X_NOUN_x', 'first_X_X_x', 'first_NOUN_VERB_x', 'last_NOUN_VERB_x', 'first_NOUN_X_x', 'last_NOUN_X_x', 'last_NUM_NOUN_x', 'first_PART_VERB_x', 'last_PART_VERB_x', 'first_NUM_NOUN_x', 'last_NOUN_PART_x', 'last_NOUN_NOUN_x', 'first_NOUN_PART_x', 'first_PART_X_x', 'first_PRON_X_x', 'last_PRON_X_x', 'first_VERB_ADJ_x', 'last_VERB_ADJ_x', 'first_VERB_ADP_x', 'last_VERB_ADP_x', 'last_PRON_PRON_x', 'first_PRON_NOUN_x', 'last_PRON_NOUN_x', 'first_PRON_PRON_x', 'last_NOUN_ADV_x', 'last_NUM_NOUN_y', 'first_ADP_ADJ_x', 'last_ADJ_X_x', 'last_ADJ_NOUN_x', 'fPOS_CONJ_y', 'fPOS_ADV_y', 'last_ADV_ADV_x', 'last_ADP_NOUN_x', 'first_ADP_NOUN_x', 'last_ADP_ADJ_x', 'fPOS_ADJ_y', 'first_ADJ_NOUN_y', 'first_ADJ_X_y', 'first_ADP_ADJ_y', 'last_ADP_ADJ_y', 'first_ADP_NOUN_y', 'last_ADP_NOUN_y', 'first_ADV_NOUN_y', 'Case_Ins_x', 'first_ADV_VERB_y', 'first_ADV_X_y', 'last_ADV_X_y', 'first_CONJ_NOUN_y', 'first_CONJ_X_y', 'first_NOUN_ADJ_y', 'last_NOUN_ADJ_y', 'first_NOUN_ADV_y', 'last_NOUN_NOUN_y', 'first_NOUN_VERB_y', 'first_NOUN_X_y', 'last_NOUN_X_y', 'first_ADV_ADV_y', 'Voice_Pass_y', 'VerbForm_Inf_y', 'VerbForm_Ger_y', 'fPOS_PRON_x', 'fPOS_ADV_x', 'Comparision_Sup_y', 'last_CONJ_X_x', 'last_ADV_X_x', 'first_CONJ_NOUN_x', 'first_ADV_X_x', 'first_CONJ_X_x', 'first_NOUN_ADV_x', 'last_NOUN_ADJ_x', 'last_CONJ_NOUN_x', 'first_NOUN_ADJ_x', 'last_PART_X_x', 'last_CONJ_ADV_y', 'last_ADJ_X_y', 'last_ADV_ADV_y', 'last_VERB_X_x', 'last_NOUN_ADV_y', 'first_CONJ_ADV_y', 'last_X_X_x', 'last_VERB_ADV_y', 'last_VERB_PRON_y', 'last_NOUN_PART_y', 'first_ADV_VERB_x', 'fPOS_VERB_y', 'last_ADJ_NOUN_y', 'first_ADV_ADV_x', 'fPOS_ADP_y', 'first_VERB_ADV_x', 'first_NOUN_NOUN_y', 'morph_correlation']
# textual = ['\\bтам_count_X', '\\bв\\b_count_X', '\\bеще_count_X', 'полагаем_count_X', '\\bпока\\b_count_X', 'поз(?:дн|же)_count_X', 'позволяющ_count_X', 'позволя_count_X', 'подход_count_X', 'последн_count_X', 'подразумевает_count_X', '\\bпод\\b_count_X', '\\bпо\\b_count_X', 'переда_count_X', 'перед_count_X', 'первоначально_count_X', 'ошибкой_count_X', 'очередь_count_X', 'получаем_count_X', 'поможет_count_X', 'порожда_count_X', 'при условии_count_X', 'прежде_count_X', 'проявля_count_X', 'предъяв_count_X', 'представ_count_X', 'предназначен_count_X', 'пределами_count_X', 'правда_count_X', 'похож(?:ий|ая|ие|ее)_count_X', 'послужи_count_X', 'порожд_count_X', 'отчасти_count_X', 'обстоятельств_count_X', 'образцы_count_X', 'обнаружил_count_X', 'обнаружи_count_X', 'обеспечивающ_count_X', 'обеспечива_count_X', 'обвини_count_X', '\\bоб?\\b_count_X', '\\bно и\\b_count_X', '\\bних\\b_count_X', 'неясн_count_X', 'несомненн_count_X', 'несмотря_count_X', 'неразумн_count_X', 'необходим_count_X', 'обусловл_count_X', 'объясн_count_X', 'обязательн_count_X', 'отреагирова_count_X', 'отправной_count_X', 'отмеч_count_X', 'отмети_count_X', 'отместку_count_X', 'отличи_count_X', '\\bот\\b_count_X', 'приводи_count_X', 'особ_count_X', 'определ_count_X', 'оправдыва_count_X', 'опасаться_count_X', 'одним_count_X', '(?:один|одна|одно|одни)_count_X', 'особенн_count_X', 'призна_count_X', '\\bстать\\b_count_X', '\\bстав\\b_count_X', 'состоять_count_X', 'составе_count_X', 'сопровождении_count_X', 'сомневаться_count_X', 'совместно_count_X', 'случа_count_X', 'слову_count_X', '\\bисходя\\b_count_X', 'слишком_count_X', 'следующим_count_X', 'степени_count_X', '\\bтон\\b_count_X', '\\bтому\\b_count_X', 'только_count_X', 'толчком_count_X', 'теперь_count_X', '\\bтак(?:ие|им|их|ов|ой|ая|ое)\\b_count_X', '\\bтакже\\b_count_X', 'считать,_count_X', 'считать_count_X', 'счету_count_X', '\\bсути\\b_count_X', '\\bсудя\\b_count_X', 'тогда_count_X', 'немедленн_count_X', '\\bследу_count_X', '\\bследстви_count_X', 'район_count_X', 'разумеется_count_X', 'разделя_count_X', 'разве_count_X', 'равно_count_X', 'пути_count_X', 'пусть_count_X', 'противоположн_count_X', 'против_count_X', 'просто_count_X', 'проблемы_count_X', 'причем_count_X', 'приходится_count_X', 'пример_count_X', 'применительно_count_X', 'ранее_count_X', 'скорее_count_X', 'сказаться_count_X', 'сказать,_count_X', 'связан_count_X', 'сведениям_count_X', '\\bс тем\\b_count_X', 'ряде_count_X', 'роль_count_X', 'речь_count_X', 'рассказ(?:а|ыва)_count_X', '\\bсам[ое]_count_X', 'вместо_count_X', 'включи_count_X', '\\bвидно\\b_count_X', '\\bвидимо\\b_count_X', '\\bвид.?\\b_count_X', 'ввиду_count_X', '\\bважн_count_X', '\\bв чем\\b_count_X', '\\bв целом_count_X', '\\bбыл\\b_count_X', '\\bбы\\b_count_X', 'будь_count_X', 'будучи_count_X', 'больш[оаи]_count_X', 'гарантирова_count_X', 'вышесказан_count_X', 'высказ(ыв)?ать_count_X', 'вывод[ау]_count_X', '\\bвывод\\b_count_X', 'выби?ра_count_X', '\\bвтор_count_X', 'всяком_count_X', 'вследствие_count_X', '\\bвслед\\b_count_X', 'вскоре_count_X', 'всего_count_X', 'вряд(?: ли)_count_X', '\\bвот\\b_count_X', 'возникающ_count_X', 'возможность_count_X', 'бесспорно_count_X', '\\(_count_X', '\\bэто_count_X', '\\bтом\\b_count_X', '\\bтем\\b_count_X', '\\bте_count_X', '\\bта_count_X', '\\bсо\\b_count_X', '\\bситуаци_count_X', '\\bсамым\\b_count_X', '\\bс\\b_count_X', '\\bрода\\b_count_X', '\\bнего_count_X', '\\bза\\b_count_X', '\\bже\\b_count_X', '\\(.+\\)_count_X', '\\)_count_X', '\\bаналогичн_count_X', '\\bанализ\\b_count_X', '\\bа\\b_count_X', '\\bзаканчива_count_X', '^[-—]_count_X', '\\bгде\\b_count_X', '«_count_X', '\\[_count_X', '\\?_count_X', ':_count_X', '\\\\-_count_X', ',$_count_X', 'где [a-zа-я ]+ [—-]_count_X', '\\bговори.?\\b_count_X', '\\bмере\\b_count_X', 'между_count_X', 'маловероятн_count_X', '\\bлица\\b_count_X', 'линии_count_X', 'кроме э?того_count_X', 'котор_count_X', 'концов_count_X', 'комментари_count_X', '\\bко(?:й|ю|я|ем)\\b_count_X', 'когда_count_X', '\\bко?\\b_count_X', 'как(?:ая|ой|ое|ие)_count_X', '\\bих\\b_count_X', 'месте_count_X', 'мире_count_X', '\\bне только_count_X', 'настаива_count_X', 'например_count_X', 'напомни_count_X', 'написа_count_X', 'намека_count_X', 'надеяться_count_X', '\\bна\\b_count_X', 'мотиваци(?:я|ей)_count_X', 'момента_count_X', 'можно_count_X', 'может_count_X', '\\bмог_count_X', 'неизменно_count_X', 'иными?_count_X', 'доказыв_count_X', 'довольно_count_X', 'добавля_count_X', 'добави_count_X', '\\bдо\\b_count_X', 'для\\b_count_X', 'делят_count_X', 'делать_count_X', '\\bдела.т\\b_count_X', '\\bдела_count_X', 'действительно_count_X', 'дальше_count_X', '\\bдал\\b_count_X', '\\bдавать\\b_count_X', '\\bдава_count_X', 'грани_count_X', 'говорят?_count_X', 'допустим_count_X', '\\bему\\b_count_X', 'есть_count_X', 'информаци_count_X', 'интерес\\b_count_X', 'ин[?:ое|ой|ая]_count_X', 'иногда_count_X', 'иначе_count_X', 'имени_count_X', '\\bили\\b_count_X', 'известно_count_X', 'исключением_count_X', '\\bиз\\b_count_X', 'здесь_count_X', 'затем_count_X', 'застав[ли]_count_X', 'завери_count_X', 'очевидно_count_X', '\\bлибо\\b_count_X', 'невзирая_count_X', 'нетрудн_count_X', '\\._count_X', 'заключа_count_X', 'соответств_count_X', 'наряду_count_X', 'сдела_count_X', 'следовательно_count_X', 'подчерк_count_X', 'результат_count_X', 'объект_count_X', 'свете_count_X', 'писа_count_X', 'сыграть_count_X', '\\bведь\\b_count_X', 'противном_count_X', 'привести_count_X', 'из-за_count_X', 'столько_count_X', 'раздел_count_X', 'некотор_count_X', '(?:то есть|т\\\\. ?е\\\\.)_count_X', 'объяви_count_X', 'сказа_count_X', 'заяв(?:и|ля)_count_X', '\\bодной\\b_count_X', '\\bа с\\b_count_X', 'поэтому_count_X', '\\bвключая\\b_count_X', 'логично_count_X', '\\bне[е|ё]_count_X', 'получа_count_X', 'названи(?:ем|ям|ю)_count_X', 'свидетельств_count_X', 'ради_count_X', '\\bмомент\\b_count_X', 'назва_count_X', 'начн_count_X', 'включа_count_X', 'почему_count_X', 'отчего_count_X', 'причин_count_X', '\\?$_count_X', 'име(?:я|ем|ет|ют)_count_X', 'настоящ_count_X', 'способств_count_X', '\\bвидим\\b_count_X', '\\bобщ[иеу]_count_X', '\\bтой\\b_count_X', 'впрочем_count_X', 'больше_count_X', 'во многом_count_X', 'источником_count_X', 'насколько_count_X', '- ?_count_X', 'имя_count_X', 'получим_count_X', 'потребова_count_X', 'откуда_count_X', 'завис[и|я]_count_X', 'впоследствии_count_X', ',_count_X', 'преддверии_count_X', 'обозначи_count_X', 'главе_count_X', 'значит_count_X', 'потому_count_X', 'ранн(?:ий|яя|ее|ие)_count_X', 'описывать_count_X', '\\bисточник\\b_count_X', 'сообщи_count_X', 'помо[чщ]_count_X', 'отсюда_count_X', 'защиту_count_X', '=_count_X', 'предпочтительн_count_X', '\\bдело\\b_count_X', 'распоряж_count_X', 'протяжении_count_X', 'согласиться_count_X', 'невозможн_count_X', 'менее_count_X', 'вед[еу]т_count_X', '\\bво\\b_count_X', 'следует_count_X', '\\bибо\\b_count_X', '\\bим\\b_count_X', 'наглядн_count_X', '\\bи если_count_X', 'сообща_count_X', 'напротив_count_X', 'должны_count_X', 'процессе_count_X', 'задолго_count_X', 'значени_count_X', 'выра(?:зи|жа)ть_count_X', '\\bб\\b_count_X', 'интересн_count_X', '\\bв частности_count_X', 'однако_count_X', '\\bи\\b_count_X', '\\bбез\\b_count_X', 'зависимост_count_X', 'пожалуй,_count_X', 'получа_count_Y', 'позволя_count_Y', 'подчерк_count_Y', 'подтвержд_count_Y', 'подачи_count_Y', '\\bпод\\b_count_Y', '\\bпо\\b_count_Y', 'первоначально_count_Y', '\\bперв_count_Y', 'ошибкой_count_Y', 'очевидно_count_Y', 'приведе_count_Y', '\\bи при\\b_count_Y', 'проявля_count_Y', 'предъяв_count_Y', 'представ_count_Y', 'предпочтительн_count_Y', 'предположив_count_Y', 'предмет_count_Y', 'преддверии_count_Y', 'правах_count_Y', 'почему_count_Y', 'похож(?:ий|ая|ие|ее)_count_Y', 'потребова_count_Y', 'послужи_count_Y', 'порожда_count_Y', 'оценк_count_Y', 'отреагирова_count_Y', 'образом_count_Y', 'обнаружи_count_Y', 'обвини_count_Y', 'нужно_count_Y', '\\bних\\b_count_Y', 'ниве_count_Y', 'несмотря_count_Y', 'неразумн_count_Y', 'необходим_count_Y', 'немедленн_count_Y', 'обстоятельств_count_Y', 'отправной_count_Y', 'отместку_count_Y', 'отличи_count_Y', 'откуда_count_Y', '\\bот\\b_count_Y', 'особенн_count_Y', 'особ_count_Y', 'основ_count_Y', 'определ_count_Y', 'описывать_count_Y', 'однако_count_Y', '(?:один|одна|одно|одни)_count_Y', 'приводи_count_Y', 'применив_count_Y', 'ср\\._count_Y', 'состоять_count_Y', 'состоит_count_Y', 'сопровождении_count_Y', 'сообщи_count_Y', 'сомневаться_count_Y', 'согласиться_count_Y', 'смену_count_Y', 'см\\._count_Y', 'слову_count_Y', '\\bисходя\\b_count_Y', '\\bсловам\\b_count_Y', 'следующим_count_Y', '\\bстать\\b_count_Y', 'стоит_count_Y', 'столь_count_Y', 'шагом_count_Y', 'следующ_count_Y', '\\bтех\\b_count_Y', '\\bтак(?:ие|им|их|ов|ой|ая|ое)\\b_count_Y', '\\bтак\\b_count_Y', '\\bсчет\\b_count_Y', 'разумеется_count_Y', 'разделя_count_Y', 'ради_count_Y', 'пусть_count_Y', 'протяжении_count_Y', 'противоположн_count_Y', 'противном_count_Y', 'проблемы_count_Y', 'приходится_count_Y', 'применительно_count_Y', 'рамках_count_Y', 'ранее_count_Y', 'ранн(?:ий|яя|ее|ие)_count_Y', 'сказаться_count_Y', 'сказать,_count_Y', 'сказа_count_Y', '\\bсилу\\b_count_Y', 'связан_count_Y', 'свидетельств_count_Y', 'свете_count_Y', 'сведениям_count_Y', '\\bследстви_count_Y', 'ряде_count_Y', 'роль_count_Y', 'роли_count_Y', 'решение_count_Y', 'результат_count_Y', 'рассчитывая_count_Y', 'рассм[ао]тр_count_Y', 'неимением_count_Y', 'независим_count_Y', 'вместе_count_Y', 'включи_count_Y', 'включа_count_Y', '\\bвидимо\\b_count_Y', '\\bвид.?\\b_count_Y', 'вед[еу]т_count_Y', '\\bважн_count_Y', '\\bв [а-я]+[еи]_count_Y', '\\bбы\\b_count_Y', 'будучи_count_Y', 'буд[еу][мт]_count_Y', 'вместо_count_Y', '\\bбок_count_Y', 'во?-[а-я]+х_count_Y', 'возможность_count_Y', 'гарантирова_count_Y', 'вышесказан_count_Y', 'выполня[ею]тся_count_Y', 'вызы?ва_count_Y', '\\bвтор_count_Y', 'всяком_count_Y', '\\bвслед\\b_count_Y', 'вряд(?: ли)_count_Y', 'впоследствии_count_Y', '\\bвот\\b_count_Y', 'безусловно_count_Y', '\\bбез\\b_count_Y', '"_count_Y', '(?:то есть|т\\\\. ?е\\\\.)_count_Y', '\\bтам_count_Y', '\\bсо\\b_count_Y', '\\bсамым\\b_count_Y', '\\bрода\\b_count_Y', '\\bобщ[иеу]_count_Y', '\\bо\\b_count_Y', '\\bим\\b_count_Y', '\\bже\\b_count_Y', '\\bеще_count_Y', '\\(_count_Y', '\\(.+\\)_count_Y', '\\bаналогичн_count_Y', '\\bанализ\\b_count_Y', '\\bа\\b_count_Y', '\\bто\\b_count_Y', '\\bпосле\\b_count_Y', '\\bзаканчива_count_Y', '^[-—]_count_Y', ',$_count_Y', '\\[_count_Y', ':_count_Y', '\\d+_count_Y', '\\.\\.\\._count_Y', '- ?_count_Y', '\\\\-_count_Y', '\\+_count_Y', 'где [a-zа-я ]+ [—-]_count_Y', '\\bмере\\b_count_Y', 'менее_count_Y', 'между_count_Y', '\\bмало\\b_count_Y', '\\bлишь\\b_count_Y', 'линии_count_Y', 'котор_count_Y', 'концов_count_Y', '\\bко(?:й|ю|я|ем)\\b_count_Y', '\\bкак\\b_count_Y', 'источником_count_Y', 'месте_count_Y', 'мире_count_Y', '\\bне только_count_Y', 'начина_count_Y', 'настаива_count_Y', 'насколько_count_Y', 'наряду_count_Y', 'наоборот_count_Y', '\\bисточник\\b_count_Y', '\\bнам\\b_count_Y', 'надеяться_count_Y', 'наглядн_count_Y', 'момента_count_Y', 'можно_count_Y', 'может_count_Y', 'назва_count_Y', 'исключением_count_Y', 'добави_count_Y', 'для\\b_count_Y', 'делят_count_Y', '\\bдело\\b_count_Y', 'делать_count_Y', 'дальше_count_Y', '\\bдал\\b_count_Y', 'говорить?ся_count_Y', 'должны_count_Y', 'интересн_count_Y', 'иногда_count_Y', 'име(?:я|ем|ет|ют)_count_Y', '\\bиз\\b_count_Y', '\\bи если_count_Y', 'зрения_count_Y', 'заяв(?:и|ля)_count_Y', 'защиту_count_Y', 'затем_count_Y', 'застав[ли]_count_Y', 'заме(?:ти|чать)_count_Y', 'есть_count_Y', '\\bибо\\b_count_Y', '\\bситуаци_count_Y', 'мнени_count_Y', '\\bвсе\\b_count_Y', 'большинство_count_Y', '\\d+[)\\.#]_count_Y', 'бесспорно_count_Y', '\\bлибо\\b_count_Y', 'оказ_count_Y', 'причин_count_Y', '\\bстановит_count_Y', '\\bнего_count_Y', 'раздел_count_Y', 'очередь_count_Y', 'повыс_count_Y', 'подобным_count_Y', 'мочь_count_Y', 'несомненн_count_Y', 'наблюдени_count_Y', '\\bгде\\b_count_Y', 'образцы_count_Y', 'вести_count_Y', '\\?$_count_Y', 'завери_count_Y', 'невзирая_count_Y', 'тогда_count_Y', 'пути_count_Y', '\\bко?\\b_count_Y', '\\bта_count_Y', '\\bслова\\b_count_Y', 'толчком_count_Y', 'обнаружил_count_Y', 'качестве_count_Y', 'обеспечива_count_Y', '\\bтом(?:\\),|,|..)_count_Y', 'иначе_count_Y', 'задолго_count_Y', 'подобн_count_Y', 'подразумевает_count_Y', '\\bдо\\b_count_Y', '\\bтем\\b_count_Y', 'считать,_count_Y', 'объект_count_Y', 'потому_count_Y', 'совместно_count_Y', 'распоряж_count_Y', '\\bследу_count_Y', 'значени_count_Y', 'во многом_count_Y', 'обеспечивающ_count_Y', 'отсюда_count_Y', 'данн_count_Y', 'возникающ_count_Y', '\\bсам[ое]_count_Y', '\\bему\\b_count_Y', '\\bна\\b_count_Y', '\\bтому\\b_count_Y', 'показ_count_Y', '\\bмог_count_Y', 'именно_count_Y', 'порожд_count_Y', 'повод_count_Y', 'довольно_count_Y', 'согласно_count_Y', 'отмети_count_Y', 'считать_count_Y', 'далее_count_Y', 'только_count_Y', '\\bитог.\\b_count_Y', 'ключев_count_Y', 'добавля_count_Y', '\\bб\\b_count_Y', 'итак_count_Y', 'нетрудн_count_Y', '\\bа с\\b_count_Y', '\\bтам\\b_count_Y', 'отчасти_count_Y', 'столько_count_Y', 'притом_count_Y', 'прошествии_count_Y', 'маловероятн_count_Y', 'вывод[ау]_count_Y', 'напротив_count_Y', '\\bно и\\b_count_Y', 'действительно_count_Y', '\\bих\\b_count_Y', '\\bвключая\\b_count_Y', 'имени_count_Y', 'означа_count_Y', '\\bно\\b_count_Y', 'параллельно_count_Y', 'последн_count_Y', '\\bпор\\b_count_Y', 'некотор_count_Y', 'скорее_count_Y', ';_count_Y', 'оправдыва_count_Y', 'полагаем_count_Y', 'изменя_count_Y', 'призна_count_Y', 'пос[кт]ольку_count_Y', '\\bте_count_Y', 'просто_count_Y', '\\)_count_Y', '\\bмы\\b_count_Y', '\\bлица\\b_count_Y', '\\bв целом_count_Y', 'впрочем_count_Y', 'обозначи_count_Y', '\\bвывод\\b_count_Y', 'отношении_count_Y', '\\bн[е|и]\\b_count_Y', 'получим_count_Y', 'сравнению_count_Y', 'обязательн_count_Y', '\\bпока\\b_count_Y', 'интерес\\b_count_Y', 'следует_count_Y', 'как(?:ая|ой|ое|ие)_count_Y', 'ввиду_count_Y', 'first_pair_помога_y', 'first_pair_одновременно_y', 'first_pair_после_y', 'first_pair_;_y', 'first_pair_подчерк_y', 'first_pair_\\bн[е|и]\\b_y', 'first_pair_\\bмог_y', 'first_pair_подобн_y', 'first_pair_\\bитог.\\b_y', 'first_pair_:_y', 'first_pair_,_y', 'first_pair_\\bих\\b_y', 'first_pair_\\bсилу\\b_y', 'first_pair_всяком_y', 'first_pair_\\bтак\\b_y', 'first_pair_\\bзаканчива_y', 'first_pair_потому_y', 'first_pair_дальше_y', 'first_pair_\\bточк(?:и|ой)_y', 'first_pair_\\bиз\\b_y', 'first_pair_поз(?:дн|же)_y', 'first_pair_сведениям_y', 'first_pair_значени_y', 'first_pair_имя_y', 'first_pair_врем_y', 'first_pair_\\bмере\\b_y', 'first_pair_име(?:я|ем|ет|ют)_y', 'first_pair_\\bа\\b_y', 'first_pair_,$_y', 'first_pair_^[-—]_y', 'first_pair_\\bко?\\b_y', 'first_pair_стоит_y', 'first_pair_заключа_y', 'first_pair_\\bпо\\b_y', 'first_pair_начина_y', 'first_pair_перед_y', 'first_pair_соответств_y', 'first_pair_начал[аеоу]_y', 'first_pair_\\bнам\\b_y', 'first_pair_добави_y', 'first_pair_\\bговори.?\\b_y', 'first_pair_обеспечивающ_y', 'first_pair_район_y', 'first_pair_\\bведь\\b_y', 'first_pair_\\bна\\b_y', 'first_pair_буд[еу][мт]_y', 'first_pair_вывод[ау]_y', 'first_pair_\\bвсе\\b_y', 'first_pair_между_y', 'first_pair_\\bпод\\b_y', 'first_pair_далее_y', 'first_pair_\\bпор\\b_y', 'first_pair_обстоятельств_y', 'first_pair_здесь_y', 'first_pair_\\bбы\\b_y', 'first_pair_\\._y', 'first_pair_имени_y', 'first_pair_\\bвот\\b_y', 'first_pair_друго_y', 'first_pair_предмет_y', 'first_pair_даже_y', 'first_pair_делать_y', 'first_pair_нужно_y', 'first_pair_\\bчего\\b_y', 'first_pair_ходе_y', 'first_pair_называ_y', 'first_pair_други_y', 'first_pair_столь_y', 'first_pair_\\bисходя\\b_y', 'first_pair_\\bтот\\b_y', 'first_pair_!_y', 'first_pair_сторон_y', 'first_pair_причем_y', 'first_pair_сказа_y', 'first_pair_маловероятн_y', 'first_pair_роль_y', 'first_pair_форме_y', 'first_pair_порожда_y', 'first_pair_\\bмало\\b_y', 'first_pair_разделя_y', 'first_pair_разумеется_y', 'first_pair_"_y', 'first_pair_больше_y', 'first_pair_показ_y', 'first_pair_как(?:ая|ой|ое|ие)_y', 'first_pair_обнаружил_y', 'first_pair_откуда_y', 'first_pair_мире_y', 'first_pair_целях_y', 'first_pair_иными?_y', 'first_pair_вызы?ва_y', 'first_pair_надеяться_y', 'first_pair_кажется_y', 'first_pair_требуется_y', 'first_pair_причин_y', 'first_pair_приходится_y', 'first_pair_\\bтом\\b_y', 'first_pair_–_y', 'first_pair_\\bли\\b_y', 'first_pair_\\bчаще\\b_y', 'first_pair_\\bвидно\\b_y', 'first_pair_\\bто ?_y', 'first_pair_пос[кт]ольку_y', 'first_pair_(?:один|одна|одно|одни)_y', 'first_pair_заяв(?:и|ля)_y', 'first_pair_означа_y', 'first_pair_\\bи если_y', 'first_pair_противоположн_y', 'first_pair_\\bследстви_y', 'first_pair_\\bцель\\b_y', 'first_pair_позволя_y', 'first_pair_\\bи при\\b_y', 'first_pair_потребова_y', 'first_pair_\\bлица\\b_y', 'first_pair_\\bпока\\b_y', 'first_pair_\\bеще_y', 'first_pair_тогда_y', 'first_pair_раздел_y', 'first_pair_\\bнего_y', 'first_pair_отместку_y', 'first_pair_части_y', 'first_pair_переда_y', 'first_pair_\\bему\\b_y', 'first_pair_\\bс\\b_y', 'first_pair_разве_y', 'first_pair_направленн_y', 'first_pair_чтобы_y', 'first_pair_собственно_y', 'first_pair_\\bи\\b_y', 'first_pair_заме(?:ти|чать)_y', 'first_pair_вести_y', 'first_pair_настоящ_y', 'first_pair_намека_y', 'first_pair_\\bтут\\b_y', 'first_pair_неимением_y', 'first_pair_\\.\\\\.\\\\._y', 'first_pair_результат_y', 'first_pair_первоначально_y', 'first_pair_рамках_y', 'first_pair_\\bно\\b_y', 'first_pair_трет(?:ий|ая|ое|ьи)_y', 'first_pair_приводи_y', 'first_pair_\\bсам[ое]_y', 'first_pair_\\bчто\\b_y', 'first_pair_однако_y', 'first_pair_свидетельств_y', 'first_pair_объяви_y', 'first_pair_качестве_y', 'first_pair_ранее_y', 'first_pair_довольно_y', 'first_pair_подход_y', 'first_pair_обнаружи_y', 'first_pair_степени_y', 'first_pair_отличи_y', 'first_pair_назва_y', 'first_pair_\\(_y', 'first_pair_теперь_y', 'first_pair_может_y', 'first_pair_менее_y', 'first_pair_рассказыва_y', 'first_pair_\\bчему\\b_y', 'first_pair_должны_y', 'first_pair_писа_y', 'first_pair_\\bдава_y', 'first_pair_согласно_y', 'first_pair_\\bили\\b_y', 'first_pair_\\bте_y', 'first_pair_\\bэто[ймг]\\b_y', 'first_pair_\\bв\\b_y', 'first_pair_\\bтого\\b_y', 'first_pair_завери_y', 'first_pair_изменя_y', 'first_pair_если_y', 'first_pair_очевидно_y', 'first_pair_конечн_y', 'first_pair_скорее_y', 'first_pair_особ_y', 'first_pair_\\bтакже\\b_y', 'first_pair_информаци_y', 'first_pair_правда_y', 'first_pair_некотор_y', 'first_pair_указыва_y', 'first_pair_пути_y', 'first_pair_\\bдо\\b_y', 'first_pair_\\bвслед\\b_y', 'first_pair_\\bим\\b_y', 'first_pair_\\bтам\\b_y', 'first_pair_именно_y', 'first_pair_\\bта_y', 'first_pair_через_y', 'first_pair_\\bлибо\\b_y', 'first_pair_объект_y', 'first_pair_выби?ра_y', 'first_pair_необходим_y', 'first_pair_числ_y', 'first_pair_рассчитывая_y', 'first_pair_конце_y', 'first_pair_неясн_y', 'first_pair_\\)_y', 'first_pair_\\d+_y', 'first_pair_частности_y', 'first_pair_невозможн_y', 'first_pair_\\bуказа_y', 'first_pair_\\bэто_y', 'first_pair_повыш_y', 'first_pair_одним_y', 'first_pair_\\bгде\\b_y', 'first_pair_несмотря_y', 'first_pair_\\bрода\\b_y', 'first_pair_говорить?ся_y', 'first_pair_получа_y', 'first_pair_\\bоб?\\b_y', 'first_pair_особенн_y', 'first_pair_предназначен_y', 'first_pair_\\bтом(?:\\),|,|..)_y', 'first_pair_\\bсамым\\b_y', 'first_pair_точнее_y', 'first_pair_следующ_y', 'first_pair_решение_y', 'first_pair_\\bв [а-я]+[еи]_y', 'first_pair_образцы_y', 'first_pair_\\bтому\\b_y', 'first_pair_только_y', 'first_pair_предпочтительн_y', 'first_pair_основ_y', 'first_pair_независим_y', 'first_pair_«_y', 'first_pair_\\bчем\\b_y', 'first_pair_\\bслова\\b_y', 'first_pair_\\bявля[ею]_y', 'first_pair_всего_y', 'first_pair_ради_y', 'first_pair_притом_y', 'first_pair_когда_y', 'first_pair_направлен_y', 'first_pair_\\bдело\\b_y', 'first_pair_котор_y', 'first_pair_призна_y', 'first_pair_\\bтем[ау]\\b_y', 'first_pair_послужи_y', 'first_pair_главе_y', 'first_pair_характеризу_y', 'first_pair_утвержда_y', 'first_pair_сдела_y', 'first_pair_случа_y', 'first_pair_\\bуже\\b_y', 'first_pair_способств_y', 'first_pair_пределами_y', 'first_pair_\\+_y', 'first_pair_хотя_y', 'first_pair_\\bтой\\b_y', 'first_pair_\\bэтим\\b_y', 'first_pair_известно_y', 'first_pair_проявля_y', 'first_pair_сыграть_y', 'first_pair_равно_y', 'first_pair_отмеч_y', 'first_pair_\\bстать\\b_y', 'first_pair_отмети_y', 'first_pair_можно_y', 'first_pair_следует_y', 'first_pair_широк_y', 'first_pair_\\bмы\\b_y', 'first_pair_почему_y', 'first_pair_слишком_y', 'first_pair_\\bчто [а-я]+ет\\b_y', 'first_pair_обвини_y', 'first_pair_учетом_y', 'first_pair_отсюда_y', 'first_pair_речь_y', 'first_pair_\\bза\\b_y', 'first_pair_\\bот\\b_y', 'first_pair_\\bвид.?\\b_y', 'first_pair_составе_y', 'first_pair_застав[ли]_y', 'first_pair_пример_y', 'first_pair_\\bних\\b_y', 'first_pair_наблюдени_y', 'first_pair_интересн_y', 'first_pair_течение_y', 'first_pair_ин[?:ое|ой|ая]_y', 'first_pair_связан_y', 'first_pair_применительно_y', 'first_pair_позволяющ_y', 'first_pair_оказ_y', 'first_pair_состоять_y', 'first_pair_\\bлишь\\b_y', 'first_pair_учитыва_y', 'first_pair_больш[оаи]_y', 'first_pair_привести_y', 'first_pair_сторон_x', 'first_pair_маловероятн_x', 'first_pair_роль_x', 'first_pair_\\._x', 'first_pair_\\?_x', 'first_pair_\\bговори.?\\b_x', 'first_pair_обеспечивающ_x', 'first_pair_имени_x', 'first_pair_\\bтот\\b_x', 'first_pair_\\bисходя\\b_x', 'first_pair_добави_x', 'first_pair_\\bвот\\b_x', 'first_pair_друго_x', 'first_pair_предмет_x', 'first_pair_даже_x', 'first_pair_делать_x', 'first_pair_нужно_x', 'first_pair_\\bчего\\b_x', 'first_pair_ходе_x', 'first_pair_други_x', 'first_pair_форме_x', 'first_pair_необходим_x', 'first_pair_конце_x', 'first_pair_наоборот_x', 'first_pair_котор_x', 'first_pair_призна_x', 'first_pair_точнее_x', 'first_pair_\\bтем[ау]\\b_x', 'first_pair_главе_x', 'first_pair_характеризу_x', 'first_pair_\\bведь\\b_x', 'first_pair_\\bбы\\b_x', 'first_pair_\\bна\\b_x', 'first_pair_вывод[ау]_x', 'first_pair_\\bвсе\\b_x', 'first_pair_зрения_x', 'first_pair_\\bже\\b_x', 'first_pair_\\bпор\\b_x', 'first_pair_обстоятельств_x', 'first_pair_\\bто\\b_x', 'first_pair_\\bко?\\b_x', 'first_pair_стоит_x', 'first_pair_заключа_x', 'first_pair_\\bтех\\b_x', 'first_pair_начина_x', 'first_pair_перед_x', 'first_pair_соответств_x', 'first_pair_^[-—]_x', 'first_pair_настоящ_x', 'first_pair_намека_x', 'first_pair_столь_x', 'first_pair_значени_x', 'first_pair_имя_x', 'first_pair_врем_x', 'first_pair_име(?:я|ем|ет|ют)_x', 'first_pair_\\bа\\b_x', 'first_pair_есть_x', 'first_pair_,_x', 'first_pair_\\bих\\b_x', 'first_pair_\\bсилу\\b_x', 'first_pair_всяком_x', 'first_pair_\\bвывод\\b_x', 'first_pair_потому_x', 'first_pair_объясн_x', 'first_pair_\\bточк(?:и|ой)_x', 'first_pair_\\bитог.\\b_x', 'first_pair_сведениям_x', 'first_pair_после_x', 'first_pair_;_x', 'first_pair_повод_x', 'first_pair_подчерк_x', 'first_pair_\\bмог_x', 'first_pair_помо[чщ]_x', 'first_pair_подобн_x', 'first_pair_\\bсамым\\b_x', 'first_pair_\\bсвязи\\b_x', 'first_pair_можно_x', 'first_pair_\\bте_x', 'first_pair_\\bили\\b_x', 'first_pair_широк_x', 'first_pair_\\bэто[ймг]\\b_x', 'first_pair_\\bмы\\b_x', 'first_pair_почему_x', 'first_pair_\\bсо\\b_x', 'first_pair_рассм[ао]тр_x', 'first_pair_привести_x', 'first_pair_связан_x', 'first_pair_позволяющ_x', 'first_pair_изменя_x', 'first_pair_\\bстать\\b_x', 'first_pair_состоять_x', 'first_pair_завери_x', 'first_pair_\\bтого\\b_x', 'first_pair_\\bв\\b_x', 'first_pair_отмеч_x', 'first_pair_\\bв [а-я]+[еи]_x', 'first_pair_\\bвслед\\b_x', 'first_pair_\\bим\\b_x', 'first_pair_против_x', 'first_pair_\\bтам\\b_x', 'first_pair_может_x', 'first_pair_именно_x', 'first_pair_просто_x', 'first_pair_\\bта_x', 'first_pair_скорее_x', 'first_pair_\\bлибо\\b_x', 'first_pair_проблемы_x', 'first_pair_пути_x', 'first_pair_решение_x', 'first_pair_\\?$_x', 'first_pair_порожд_x', 'first_pair_особ_x', 'first_pair_писа_x', 'first_pair_\\bтакже\\b_x', 'first_pair_информаци_x', 'first_pair_некотор_x', 'first_pair_должны_x', 'first_pair_ид(?:ет|ти)_x', 'first_pair_наблюдени_x', 'first_pair_\\bважн_x', 'first_pair_\\bгде\\b_x', 'first_pair_\\bэтим\\b_x', 'first_pair_значит_x', 'first_pair_\\bдело\\b_x', 'first_pair_говорят?_x', 'first_pair_повыш_x', 'first_pair_образцы_x', 'first_pair_\\bтому\\b_x', 'first_pair_ниве_x', 'first_pair_отношении_x', 'first_pair_\\bвопрос_x', 'first_pair_\\bвключая\\b_x', 'first_pair_несмотря_x', 'first_pair_утвержда_x', 'first_pair_сдела_x', 'first_pair_особенн_x', 'first_pair_\\bоб?\\b_x', 'first_pair_получа_x', 'first_pair_\\bвтор_x', 'first_pair_способств_x', 'first_pair_пределами_x', 'first_pair_\\+_x', 'first_pair_\\bуже\\b_x', 'first_pair_предпочтительн_x', 'first_pair_интересн_x', 'first_pair_\\bисточник\\b_x', 'first_pair_объект_x', 'first_pair_\\bза\\b_x', 'first_pair_составе_x', 'first_pair_застав[ли]_x', 'first_pair_пример_x', 'first_pair_\\bних\\b_x', 'first_pair_\\bот\\b_x', 'first_pair_сыграть_x', 'first_pair_\\bуказа_x', 'first_pair_независим_x', 'first_pair_«_x', 'first_pair_\\bчем\\b_x', 'first_pair_\\bкак\\b_x', 'first_pair_\\bслова\\b_x', 'first_pair_\\bявля[ею]_x', 'first_pair_сообщи_x', 'first_pair_притом_x', 'first_pair_частности_x', 'first_pair_когда_x', 'first_pair_\\d+_x', 'first_pair_\\bтом(?:\\),|,|..)_x', 'first_pair_!_x', 'first_pair_вести_x', 'first_pair_кажется_x', 'first_pair_надеяться_x', 'first_pair_целях_x', 'first_pair_откуда_x', 'first_pair_обнаружил_x', 'first_pair_как(?:ая|ой|ое|ие)_x', 'first_pair_определ_x', 'first_pair_"_x', 'first_pair_роли_x', 'first_pair_\\bсути\\b_x', 'first_pair_показ_x', 'first_pair_\\bмало\\b_x', 'first_pair_оценк_x', 'first_pair_причин_x', 'first_pair_ранее_x', 'first_pair_объяви_x', 'first_pair_\\bсам[ое]_x', 'first_pair_приводи_x', 'first_pair_\\bно\\b_x', 'first_pair_распоряж_x', 'first_pair_результат_x', 'first_pair_\\.\\\\.\\\\._x', 'first_pair_\\bеще_x', 'first_pair_трет(?:ий|ая|ое|ьи)_x', 'first_pair_довольно_x', 'first_pair_\\bли\\b_x', 'first_pair_\\bчто и\\b_x', 'first_pair_–_x', 'first_pair_\\bбыл\\b_x', 'first_pair_проявля_x', 'first_pair_\\bмомент\\b_x', 'first_pair_- ?_x', 'first_pair_\\bтем\\b_x', 'first_pair_\\bчаще\\b_x', 'first_pair_приходится_x', 'first_pair_\\bлица\\b_x', 'first_pair_\\bтом\\b_x', 'first_pair_более_x', 'first_pair_\\bфакт\\b_x', 'first_pair_означа_x', 'first_pair_заяв(?:и|ля)_x', 'first_pair_(?:один|одна|одно|одни)_x', 'first_pair_\\bобщ[иеу]_x', 'first_pair_пос[кт]ольку_x', 'first_pair_\\bвидно\\b_x', 'first_pair_пусть_x', 'first_pair_направленн_x', 'first_pair_неимением_x', 'first_pair_\\bи\\b_x', 'first_pair_чтобы_x', 'first_pair_\\bнего_x', 'first_pair_\\bтак(?:ие|им|их|ов|ой|ая|ое)\\b_x', 'first_pair_\\bс\\b_x', 'first_pair_собственно_x', 'first_pair_степени_x', 'first_pair_переда_x', 'first_pair_обнаружи_x', 'first_pair_тогда_x', 'first_pair_части_x', 'first_pair_действительно_x', 'first_pair_подход_x', 'first_pair_отместку_x', 'first_pair_\\bтут\\b_x', 'first_pair_услови_y', 'first_pair_грани_y', 'first_pair_\\bрода\\b_x', 'first_pair_вместо_y', 'first_pair_объясн_y', 'first_pair_пользу_y', 'first_pair_ошибкой_y', 'first_pair_говорят?_y', 'first_pair_сообщи_y', 'first_pair_\\bэто\\b_y', 'first_pair_мнени_x', 'first_pair_разве_x', 'first_pair_\\bвтор_y', 'first_pair_смену_y', 'first_pair_район_x', 'first_pair_повод_y', 'first_pair_равно_x', 'first_pair_ин[?:ое|ой|ая]_x', 'first_pair_\\bчто и\\b_y', 'first_pair_есть_y', 'first_pair_помога_x', 'first_pair_\\bэто\\b_x', 'first_pair_\\bкак\\b_y', 'first_pair_\\bситуаци_x', 'first_pair_учитыва_x', 'first_pair_\\bн[е|и]\\b_x', 'first_pair_помо[чщ]_y', 'first_pair_\\bпока\\b_x', 'first_pair_применительно_x', 'first_pair_\\bто ?_x', 'first_pair_просто_y', 'first_pair_\\bибо\\b_x', 'first_pair_порожд_y', 'first_pair_сообща_y', 'first_pair_распоряж_y', 'first_pair_\\bисточник\\b_y', 'first_pair_разделя_x', 'first_pair_выби?ра_x', 'first_pair_против_y', 'first_pair_\\bчему\\b_x', 'first_pair_слишком_x', 'first_pair_числ_x', 'first_pair_\\bвид.?\\b_x', 'first_pair_игра_x', 'first_pair_вызы?ва_x', 'first_pair_одним_x', 'first_pair_говорить?ся_x', 'first_pair_рамках_x', 'first_pair_наоборот_y', 'first_pair_благодаря_y', 'first_pair_рассм[ао]тр_y', 'first_pair_данн_y', 'first_pair_называ_x', 'first_pair_\\bсвязи\\b_y', 'first_pair_данн_x', 'first_pair_далее_x', 'first_pair_\\bэто_x', 'first_pair_послужи_x', 'first_pair_\\bситуаци_y', 'first_pair_рассчитывая_x', 'first_pair_отличи_x', 'first_pair_\\bиз\\b_x', 'first_pair_затем_x', 'first_pair_случа_x', 'first_pair_для\\b_x', 'first_pair_\\bдо\\b_x', 'first_pair_\\bчто [а-я]+ет\\b_x', 'first_pair_\\bбез\\b_y', 'first_pair_\\bстав\\b_x', 'first_pair_\\bсловам\\b_y', 'first_pair_через_x', 'first_pair_\\bбез\\b_x', 'first_pair_уточня_y', 'first_pair_раздел_x', 'first_pair_\\bтой\\b_x', 'first_pair_\\bперв_y', 'first_pair_роли_y', 'first_pair_\\bтак(?:ие|им|их|ов|ой|ая|ое)\\b_y', 'first_pair_\\bзаканчива_x', 'first_pair_\\bда\\b_x', 'first_pair_обязательн_x', 'first_pair_\\bво\\b_y', 'first_pair_грани_x', 'first_pair_ид(?:ет|ти)_y', 'first_pair_\\bсо\\b_y', 'first_pair_хотя_x', 'first_pair_зависимост_x', 'first_pair_\\?_y', 'first_pair_\\bдава_x', 'first_pair_сказа_x', 'first_pair_рассказыва_x', 'first_pair_речь_x', 'first_pair_между_x', 'first_pair_\\bсреди\\b_x', 'first_pair_известно_x', 'first_pair_\\bто\\b_y', 'first_pair_направлен_x', 'first_pair_ошибкой_x', 'first_pair_назва_x', 'first_pair_образом_y', 'first_pair_\\bво\\b_x', 'first_pair_\\(_x', 'first_pair_\\bибо\\b_y', 'first_pair_\\bтех\\b_y', 'first_pair_уточня_x', 'first_pair_однако_x', 'first_pair_\\bстав\\b_y', 'first_pair_подтвержд_y', 'first_pair_\\bдела_y', 'first_pair_теперь_x', 'first_pair_одновременно_x', 'last_pair_\\bн[е|и]\\b_y', 'last_pair_\\bитог.\\b_y', 'last_pair_,_y', 'last_pair_\\bда\\b_y', 'last_pair_\\bзаканчива_y', 'last_pair_помога_y', 'last_pair_\\bнам\\b_y', 'last_pair_\\bстав\\b_y', 'last_pair_значени_y', 'last_pair_имя_y', 'last_pair_име(?:я|ем|ет|ют)_y', 'last_pair_\\bа\\b_y', 'last_pair_^[-—]_y', 'last_pair_заключа_y', 'last_pair_смену_y', 'last_pair_начал[аеоу]_y', 'last_pair_поз(?:дн|же)_y', 'last_pair_\\bвот\\b_y', 'last_pair_\\?_y', 'last_pair_обеспечивающ_y', 'last_pair_район_y', 'last_pair_\\bна\\b_y', 'last_pair_буд[еу][мт]_y', 'last_pair_\\bпод\\b_y', 'last_pair_далее_y', 'last_pair_обстоятельств_y', 'last_pair_здесь_y', 'last_pair_наоборот_y', 'last_pair_\\._y', 'last_pair_имени_y', 'last_pair_друго_y', 'last_pair_даже_y', 'last_pair_форме_y', 'last_pair_!_y', 'last_pair_сторон_y', 'last_pair_сказа_y', 'last_pair_роль_y', 'last_pair_отличи_y', 'last_pair_столь_y', 'last_pair_\\bчто\\b_y', 'last_pair_разделя_y', 'last_pair_ошибкой_y', 'last_pair_разумеется_y', 'last_pair_определ_y', 'last_pair_как(?:ая|ой|ое|ие)_y', 'last_pair_обнаружил_y', 'last_pair_мире_y', 'last_pair_порожда_y', 'last_pair_\\bпока\\b_y', 'last_pair_\\bбыл\\b_y', 'last_pair_месте_y', 'last_pair_\\bто ?_y', 'last_pair_означа_y', 'last_pair_\\bцель\\b_y', 'last_pair_\\bперв_y', 'last_pair_\\bсловам\\b_y', 'last_pair_потребова_y', 'last_pair_образом_y', 'last_pair_\\.\\\\.\\\\._y', 'last_pair_тогда_y', 'last_pair_раздел_y', 'last_pair_\\bнего_y', 'last_pair_данн_y', 'last_pair_пусть_y', 'last_pair_заме(?:ти|чать)_y', 'last_pair_вести_y', 'last_pair_настоящ_y', 'last_pair_\\bтут\\b_y', 'last_pair_неимением_y', 'last_pair_распоряж_y', 'last_pair_\\bно\\b_y', 'last_pair_приводи_y', 'last_pair_однако_y', 'last_pair_затем_y', 'last_pair_довольно_y', 'last_pair_обнаружи_y', 'last_pair_степени_y', 'last_pair_свидетельств_y', 'last_pair_против_y', 'last_pair_должны_y', 'last_pair_\\bво\\b_y', 'last_pair_\\?$_y', 'last_pair_рассм[ао]тр_y', 'last_pair_\\bте_y', 'last_pair_\\bтого\\b_y', 'last_pair_\\bважн_y', 'last_pair_очевидно_y', 'last_pair_правда_y', 'last_pair_некотор_y', 'last_pair_указыва_y', 'last_pair_\\bдо\\b_y', 'last_pair_\\bвслед\\b_y', 'last_pair_сообща_y', 'last_pair_\\bтам\\b_y', 'last_pair_именно_y', 'last_pair_благодаря_y', 'last_pair_через_y', 'last_pair_скорее_y', 'last_pair_выби?ра_y', 'last_pair_необходим_y', 'last_pair_пользу_y', 'last_pair_рассчитывая_y', 'last_pair_неясн_y', 'last_pair_сообщи_y', 'last_pair_одним_y', 'last_pair_\\bрода\\b_y', 'last_pair_порожд_y', 'last_pair_говорить?ся_y', 'last_pair_\\bоб?\\b_y', 'last_pair_\\bтом(?:\\),|,|..)_y', 'last_pair_\\bсамым\\b_y', 'last_pair_\\bвтор_y', 'last_pair_ниве_y', 'last_pair_только_y', 'last_pair_независим_y', 'last_pair_«_y', 'last_pair_\\bслова\\b_y', 'last_pair_всего_y', 'last_pair_ради_y', 'last_pair_когда_y', 'last_pair_направлен_y', 'last_pair_случа_y', 'last_pair_образцы_y', 'last_pair_\\bвопрос_y', 'last_pair_\\bдело\\b_y', 'last_pair_характеризу_y', 'last_pair_утвержда_y', 'last_pair_сдела_y', 'last_pair_\\bуже\\b_y', 'last_pair_хотя_y', 'last_pair_\\bтой\\b_y', 'last_pair_\\bэтим\\b_y', 'last_pair_известно_y', 'last_pair_значит_y', 'last_pair_способств_y', 'last_pair_\\bза\\b_y', 'last_pair_отмеч_y', 'last_pair_отмети_y', 'last_pair_можно_y', 'last_pair_следует_y', 'last_pair_широк_y', 'last_pair_\\bситуаци_y', 'last_pair_\\bмы\\b_y', 'last_pair_\\bдела_y', 'last_pair_слишком_y', 'last_pair_\\bчто [а-я]+ет\\b_y', 'last_pair_\\bибо\\b_y', 'last_pair_\\bв [а-я]+[еи]_y', 'last_pair_равно_y', 'last_pair_отсюда_y', 'last_pair_\\bвид.?\\b_y', 'last_pair_пример_y', 'last_pair_\\bних\\b_y', 'last_pair_наблюдени_y', 'last_pair_привести_y', 'last_pair_\\bлишь\\b_y', 'last_pair_учитыва_y', 'last_pair_\\bлибо\\b_x', 'last_pair_причем_x', 'last_pair_сказа_x', 'last_pair_маловероятн_x', 'last_pair_роль_x', 'last_pair_!_x', 'last_pair_имени_x', 'last_pair_\\._x', 'last_pair_\\?_x', 'last_pair_\\bговори.?\\b_x', 'last_pair_обеспечивающ_x', 'last_pair_район_x', 'last_pair_\\bвот\\b_x', 'last_pair_даже_x', 'last_pair_называ_x', 'last_pair_други_x', 'last_pair_форме_x', 'last_pair_\\bисходя\\b_x', 'last_pair_нужно_x', 'last_pair_добави_x', 'last_pair_\\bведь\\b_x', 'last_pair_необходим_x', 'last_pair_конце_x', 'last_pair_отличи_x', 'last_pair_назва_x', 'last_pair_\\)_x', 'last_pair_\\(_x', 'last_pair_котор_x', 'last_pair_\\bтем[ау]\\b_x', 'last_pair_главе_x', 'last_pair_характеризу_x', 'last_pair_для\\b_x', 'last_pair_призна_x', 'last_pair_пользу_x', 'last_pair_буд[еу][мт]_x', 'last_pair_\\bвсе\\b_x', 'last_pair_зрения_x', 'last_pair_\\bбы\\b_x', 'last_pair_между_x', 'last_pair_\\bпод\\b_x', 'last_pair_далее_x', 'last_pair_рассчитывая_x', 'last_pair_\\bпор\\b_x', 'last_pair_обстоятельств_x', 'last_pair_здесь_x', 'last_pair_\\bже\\b_x', 'last_pair_\\bто\\b_x', 'last_pair_\\bнам\\b_x', 'last_pair_стоит_x', 'last_pair_\\bко?\\b_x', 'last_pair_начина_x', 'last_pair_смену_x', 'last_pair_соответств_x', 'last_pair_начал[аеоу]_x', 'last_pair_есть_x', 'last_pair_намека_x', 'last_pair_\\bчто\\b_x', 'last_pair_обязательн_x', 'last_pair_столь_x', 'last_pair_значени_x', 'last_pair_^[-—]_x', 'last_pair_врем_x', 'last_pair_\\bмере\\b_x', 'last_pair_име(?:я|ем|ет|ют)_x', 'last_pair_,$_x', 'last_pair_имя_x', 'last_pair_сведениям_x', 'last_pair_,_x', 'last_pair_\\bтак\\b_x', 'last_pair_\\bэто\\b_x', 'last_pair_потому_x', 'last_pair_дальше_x', 'last_pair_объясн_x', 'last_pair_\\bвывод\\b_x', 'last_pair_:_x', 'last_pair_поз(?:дн|же)_x', 'last_pair_помога_x', 'last_pair_после_x', 'last_pair_;_x', 'last_pair_повод_x', 'last_pair_\\bн[е|и]\\b_x', 'last_pair_\\bмог_x', 'last_pair_помо[чщ]_x', 'last_pair_подобн_x', 'last_pair_\\bитог.\\b_x', 'last_pair_\\bтом(?:\\),|,|..)_x', 'last_pair_\\bэто[ймг]\\b_x', 'last_pair_можно_x', 'last_pair_\\bте_x', 'last_pair_широк_x', 'last_pair_\\bситуаци_x', 'last_pair_\\bмы\\b_x', 'last_pair_рассм[ао]тр_x', 'last_pair_почему_x', 'last_pair_слишком_x', 'last_pair_\\bсо\\b_x', 'last_pair_\\bчто [а-я]+ет\\b_x', 'last_pair_\\bибо\\b_x', 'last_pair_отмети_x', 'last_pair_\\bстать\\b_x', 'last_pair_течение_x', 'last_pair_ин[?:ое|ой|ая]_x', 'last_pair_связан_x', 'last_pair_позволяющ_x', 'last_pair_если_x', 'last_pair_\\bлишь\\b_x', 'last_pair_больш[оаи]_x', 'last_pair_завери_x', 'last_pair_\\bтого\\b_x', 'last_pair_равно_x', 'last_pair_изменя_x', 'last_pair_привести_x', 'last_pair_\\bв [а-я]+[еи]_x', 'last_pair_\\bдо\\b_x', 'last_pair_\\bвслед\\b_x', 'last_pair_\\bим\\b_x', 'last_pair_должны_x', 'last_pair_может_x', 'last_pair_благодаря_x', 'last_pair_\\bта_x', 'last_pair_через_x', 'last_pair_очевидно_x', 'last_pair_скорее_x', 'last_pair_теперь_x', 'last_pair_согласно_x', 'last_pair_решение_x', 'last_pair_\\bдава_x', 'last_pair_\\?$_x', 'last_pair_\\bво\\b_x', 'last_pair_порожд_x', 'last_pair_особ_x', 'last_pair_проблемы_x', 'last_pair_ид(?:ет|ти)_x', 'last_pair_информаци_x', 'last_pair_правда_x', 'last_pair_некотор_x', 'last_pair_наблюдени_x', 'last_pair_\\bэтим\\b_x', 'last_pair_известно_x', 'last_pair_\\bдело\\b_x', 'last_pair_\\bгде\\b_x', 'last_pair_\\bтой\\b_x', 'last_pair_\\bвопрос_x', 'last_pair_образцы_x', 'last_pair_ниве_x', 'last_pair_отношении_x', 'last_pair_повыш_x', 'last_pair_только_x', 'last_pair_предназначен_x', 'last_pair_сдела_x', 'last_pair_особенн_x', 'last_pair_\\bоб?\\b_x', 'last_pair_несмотря_x', 'last_pair_\\bуже\\b_x', 'last_pair_говорить?ся_x', 'last_pair_пределами_x', 'last_pair_\\+_x', 'last_pair_\\bрода\\b_x', 'last_pair_хотя_x', 'last_pair_предпочтительн_x', 'last_pair_основ_x', 'last_pair_\\d+_x', 'last_pair_\\bисточник\\b_x', 'last_pair_выби?ра_x', 'last_pair_\\bза\\b_x', 'last_pair_объект_x', 'last_pair_интересн_x', 'last_pair_\\bот\\b_x', 'last_pair_\\bвид.?\\b_x', 'last_pair_составе_x', 'last_pair_пример_x', 'last_pair_конечн_x', 'last_pair_\\bних\\b_x', 'last_pair_сыграть_x', 'last_pair_случа_x', 'last_pair_независим_x', 'last_pair_\\bчем\\b_x', 'last_pair_\\bкак\\b_x', 'last_pair_\\bслова\\b_x', 'last_pair_\\bявля[ею]_x', 'last_pair_сообщи_x', 'last_pair_когда_x', 'last_pair_направлен_x', 'last_pair_частности_x', 'last_pair_настоящ_x', 'last_pair_требуется_x', 'last_pair_кажется_x', 'last_pair_вызы?ва_x', 'last_pair_целях_x', 'last_pair_мире_x', 'last_pair_откуда_x', 'last_pair_оценк_x', 'last_pair_как(?:ая|ой|ое|ие)_x', 'last_pair_показ_x', 'last_pair_больше_x', 'last_pair_определ_x', 'last_pair_"_x', 'last_pair_зависимост_x', 'last_pair_разумеется_x', 'last_pair_ошибкой_x', 'last_pair_довольно_x', 'last_pair_ранее_x', 'last_pair_объяви_x', 'last_pair_затем_x', 'last_pair_свидетельств_x', 'last_pair_однако_x', 'last_pair_приводи_x', 'last_pair_причин_x', 'last_pair_уточня_x', 'last_pair_трет(?:ий|ая|ое|ьи)_x', 'last_pair_\\bно\\b_x', 'last_pair_рамках_x', 'last_pair_первоначально_x', 'last_pair_результат_x', 'last_pair_\\.\\\\.\\\\._x', 'last_pair_\\bеще_x', 'last_pair_\\bпока\\b_x', 'last_pair_включа_x', 'last_pair_\\bтем\\b_x', 'last_pair_\\bли\\b_x', 'last_pair_\\bмало\\b_x', 'last_pair_\\bвидно\\b_x', 'last_pair_порожда_x', 'last_pair_\\bтом\\b_x', 'last_pair_\\bсреди\\b_x', 'last_pair_мнени_x', 'last_pair_образом_x', 'last_pair_\\bфакт\\b_x', 'last_pair_месте_x', 'last_pair_\\bи при\\b_x', 'last_pair_\\bсловам\\b_x', 'last_pair_\\bи если_x', 'last_pair_означа_x', 'last_pair_заяв(?:и|ля)_x', 'last_pair_\\bобщ[иеу]_x', 'last_pair_\\bто ?_x', 'last_pair_подход_x', 'last_pair_приходится_x', 'last_pair_\\bему\\b_x', 'last_pair_чтобы_x', 'last_pair_разве_x', 'last_pair_степени_x', 'last_pair_\\bи\\b_x', 'last_pair_пусть_x', 'last_pair_собственно_x', 'last_pair_\\bтак(?:ие|им|их|ов|ой|ая|ое)\\b_x', 'last_pair_\\bнего_x', 'last_pair_неимением_x', 'last_pair_\\bс\\b_x', 'last_pair_грани_x', 'last_pair_переда_x', 'last_pair_направленн_x', 'last_pair_действительно_x', 'last_pair_тогда_x', 'last_pair_данн_x', 'last_pair_обнаружи_x', 'last_pair_части_x', 'last_pair_\\bтак(?:ие|им|их|ов|ой|ая|ое)\\b_y', 'last_pair_други_y', 'last_pair_повод_y', 'last_pair_больш[оаи]_y', 'last_pair_всяком_y', 'last_pair_\\bтут\\b_x', 'last_pair_речь_y', 'last_pair_сыграть_y', 'last_pair_соответств_y', 'last_pair_\\bв\\b_x', 'last_pair_просто_x', 'last_pair_намека_y', 'last_pair_ради_x', 'last_pair_пос[кт]ольку_x', 'last_pair_предмет_x', 'last_pair_объясн_y', 'last_pair_\\bиз\\b_y', 'last_pair_\\bважн_x', 'last_pair_чтобы_y', 'last_pair_помо[чщ]_y', 'last_pair_говорят?_x', 'last_pair_\\bбыл\\b_x', 'last_pair_значит_x', 'last_pair_послужи_y', 'last_pair_друго_x', 'last_pair_оказ_x', 'last_pair_\\bчему\\b_x', 'last_pair_перед_y', 'last_pair_\\bстать\\b_y', 'last_pair_\\bтому\\b_x', 'last_pair_\\bдава_y', 'last_pair_\\bуказа_x', 'last_pair_способств_x', 'last_pair_включа_y', 'last_pair_позволя_y', 'last_pair_перед_x', 'last_pair_обнаружил_x', 'last_pair_следующ_x', 'last_pair_раздел_x', 'last_pair_застав[ли]_x', 'last_pair_почему_y', 'last_pair_\\bто\\b_y', 'last_pair_изменя_y', 'last_pair_\\(_y', 'last_pair_для\\b_y', 'last_pair_\\bэто_y', 'last_pair_указыва_x', 'last_pair_получа_x', 'last_pair_сведениям_y', 'last_pair_разделя_x', 'last_pair_\\bдела_x', 'last_pair_\\bчему\\b_y', 'last_pair_\\bведь\\b_y', 'last_pair_рассказыва_y', 'last_pair_причин_y', 'last_pair_интересн_y', 'last_pair_потому_y', 'last_pair_\\bстав\\b_x', 'last_pair_разве_y', 'last_pair_говорят?_y', 'last_pair_\\bко?\\b_y', 'last_pair_одним_x', 'last_pair_\\bпо\\b_x', 'last_pair_собственно_y', 'last_pair_ранее_y', 'last_pair_писа_x', 'last_pair_\\bили\\b_x', 'last_pair_\\bна\\b_x', 'last_pair_уточня_y', 'last_pair_просто_y', 'last_pair_проявля_x', 'last_pair_проблемы_y', 'last_pair_:_y', 'last_pair_\\bлица\\b_y', 'last_pair_\\bтот\\b_y', 'last_pair_кажется_y', 'last_pair_«_x', 'last_pair_\\bчего\\b_x', 'last_pair_части_y', 'last_pair_заключа_x', 'last_pair_\\bда\\b_x', 'last_pair_причем_y', 'last_pair_рамках_y', 'last_pair_ходе_x', 'last_pair_одновременно_x', 'last_pair_именно_x', 'last_pair_утвержда_x', 'last_pair_менее_y', 'last_pair_\\bмере\\b_y', 'last_pair_невозможн_y', 'last_pair_\\bих\\b_y', 'last_pair_применительно_y', 'last_pair_позволяющ_y', 'last_pair_может_y', 'last_pair_течение_y', 'last_pair_состоять_y', 'last_pair_следует_x', 'last_pair_более_y', 'last_pair_надеяться_x', 'last_pair_подобн_y', 'last_pair_отместку_x', 'last_pair_менее_x', 'last_pair_целях_y', 'last_pair_\\bбыть\\b_y', 'last_pair_\\bисточник\\b_y', 'last_pair_\\bих\\b_x', 'last_pair_\\bтех\\b_x', 'last_pair_учитыва_x', 'last_pair_предпочтительн_y', 'last_pair_точнее_x', 'last_pair_повыш_y', 'last_pair_\\bи если_y', 'last_pair_\\bбез\\b_x', 'last_pair_игра_y', 'last_pair_послужи_x', 'last_pair_отмеч_x', 'last_pair_иными?_x', 'last_pair_\\bсам[ое]_x', 'last_pair_более_x', 'last_pair_представ_y', 'last_pair_\\bвывод\\b_y', 'last_pair_надеяться_y', 'last_pair_добави_y', 'last_pair_частности_y', 'last_pair_распоряж_x', 'last_pair_\\bлибо\\b_y', 'last_pair_\\bеще_y', 'last_pair_наоборот_x', 'last_pair_\\bкак\\b_y', 'last_pair_\\bтам\\b_x', 'last_pair_всего_x', 'last_pair_\\bгде\\b_y', 'last_pair_зрения_y', 'last_pair_\\bэто_x', 'last_pair_услови_y', 'last_pair_\\bуказа_y', 'last_pair_\\bвключая\\b_x', 'last_pair_учетом_x', 'last_pair_роли_x', 'last_pair_\\bтому\\b_y', 'last_pair_\\bот\\b_y', 'last_pair_есть_y', 'last_pair_\\bа\\b_x', 'last_pair_\\bэто[ймг]\\b_y', 'last_pair_\\)_y', 'last_pair_приходится_y', 'last_pair_вести_x', 'last_pair_отместку_y', 'last_pair_\\bи\\b_y', 'last_pair_подтвержд_x', 'last_pair_всяком_x', 'last_pair_отсюда_x', 'last_pair_игра_x', 'last_pair_\\bчаще\\b_x']

# X_wo_textual = X.drop(columns=[feature for feature in textual])
# X_wo_lexical = X.drop(columns=[feature for feature in lexical])
# X_wo_morpho = X.drop(columns=[feature for feature in morphosynt])

### 3. Classifiers & CV-training

In [ ]:
lsvc = LinearSVC(random_state=random_state, C=0.0001)

bagger_lsvc = BaggingClassifier(base_estimator=lsvc, 
                                n_estimators=2, 
                                max_samples=.9, 
                                max_features=.7, 
                                bootstrap=True, 
                                bootstrap_features=False, 
                                oob_score=False, 
                                warm_start=False, 
                                n_jobs=-1, 
                                random_state=random_state, 
                                verbose=0)

logreg = LogisticRegression(random_state=random_state,
                            solver='lbfgs',
                            n_jobs=8,
                            C=0.02,
                            multi_class='auto')

lgbm_param =  {
    'tree_learner': 'feature',
    'task': 'train',
    'scale_pos_weight': 155,
    'reg_lambda': 0.11,
    'reg_alpha': 0.11,
    'random_state': random_state,
    'num_leaves': 36,
    'num_class': 11,
    'min_split_gain': 0.76,
    'min_child_weight': 0.11,
    'metric': 'multi_logloss',
    'learning_rate': 0.1,
    'feature_fraction': 0.45,
    'boosting_type': 'dart',
    'bagging_fraction': 0.9,
    'application': 'multiclass',
    'num_iterations': 1000,
}
lgb_class = lgb.LGBMClassifier(**lgbm_param)

class SklearnWrapperKerasEstimator(Model):      
    def fit(self, _X, _y):
        indices = np.arange(_y.shape[0])
        np.random.shuffle(indices)
        
        try:
            _X = _X.iloc[indices]
            _y = _y.iloc[indices]
        except AttributeError:
            _X = pd.DataFrame(_X, columns=X.columns).iloc[indices]
            _y = pd.DataFrame(_y, columns=y.columns).iloc[indices]
          
        replace_map = {TARGET : {k: v for k,v in zip(top11,list(range(0,len(top11))))}}
        y_categorical = to_categorical(_y.replace(replace_map))

        session = K.get_session()
        for layer in self.layers: 
            if hasattr(layer, 'kernel_initializer'):
                layer.kernel.initializer.run(session=session)
        
        return super(SklearnWrapperKerasEstimator, self).fit(_X, y_categorical, epochs=7, batch_size=128, validation_split=0.)
      
    def predict(self, *args, **kwargs):
        preds = super().predict(*args, **kwargs)
        preds = np.argmax(preds, axis=1)
        preds = [unique_type_list[prediction] for prediction in preds]
        return preds

    def predict_proba(self, *args, **kwargs):
        return super().predict(*args, **kwargs)
    
def construct_simple_nn_classifier(input_length, output_length):
    input_features = Input(shape=(input_length,))
    l = BatchNormalization(axis = 1)(input_features)
    l = Dense(100)(input_features)
    l = Activation('relu')(l)
    l = BatchNormalization(axis = 1)(l)
    l = Dropout(0.5)(l)
    outputs = Dense(output_length, activation='softmax')(l)

    model = SklearnWrapperKerasEstimator(inputs=[input_features], outputs=outputs)
    adam = Adam(lr=0.01, amsgrad=True)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['mean_squared_error', 'acc'])

    return model
  
#nnclassifier = construct_simple_nn_classifier(input_length=X.shape[1], output_length=len(np.unique(y)))

catboost = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.2,
    custom_loss=['F1'],
    random_seed=random_state,
    verbose=0,
    loss_function='MultiClass',
    task_type='GPU',
    class_weights=[0.55, .47, 0.5, 0.35, 0.7, 0.4, 0.25, 1., 0.6, 1., 1.]
)

fs_catboost = Pipeline([
  ('feature_selection', SelectFromModel(LogisticRegression(solver='saga', penalty='l1', C=1., n_jobs=-1))),
  ('classification', catboost)
])

fs_lgbm = Pipeline([
  ('feature_selection', SelectFromModel(LogisticRegression(penalty='l1', C=1.))),
  ('classification', lgb_class)
])

fs_catboost_plus_logreg = VotingClassifier([('fs_catboost', fs_catboost), ('logreg', LogisticRegression(random_state=random_state, C=0.1, multi_class='auto', solver='saga', penalty='l2', n_jobs=-1))], voting='soft')
fs_catboost_plus_svc = VotingClassifier([('fs_catboost', fs_catboost), ('lsvc', SVC(random_state=random_state, C=0.001, probability=True))], voting='soft')
fs_lgbm_plus_svc = VotingClassifier([('fs_lgbm', fs_lgbm), ('lsvc', SVC(random_state=random_state, C=0.001, probability=True))], voting='soft')

smt = SMOTE(random_state=random_state)
lsvc_smote = Pipeline([('smt', smt), ('lsvc', lsvc)])
logreg_smote = Pipeline([('smt', smt), ('logreg', logreg)])
#nnclassifier_smote = Pipeline([('smt', smt), ('nnclassifier', nnclassifier)])

name = lambda x : str(x).split('(')[0]

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(unique_type_list))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)
    return model
            
def train_stratified(models, X, y, add_idf=False, nsplits=5, confusion=False):
    fig_i = 0
    kf = StratifiedShuffleSplit(n_splits=nsplits, test_size=0.1, random_state=random_state)
    
    # Store folds score for each classifier in a dictionary
    dico_score = {}
    dic_report = {}

    for num, model in enumerate(models):
        dico_score[num] = [[],[],[]]  # macro_f1, micro_f1, accuracy
        dic_report[num] = []
    
    # Stratified Split
    for train, test in kf.split(X, y):
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]
        
        for num, model in enumerate(models):
            print(num, '>>', name(model))

            model.fit(X_train, y_train)

            preds = model.predict(X_test)
            macroscore = f1_score(y_test, preds, average='macro')
            print('%s macro F score: %s' % (name(model), macroscore))
            dico_score[num][0].append(macroscore)
            microscore = f1_score(y_test, preds, average='micro')
            print('%s micro F score: %s' % (name(model), microscore))
            dico_score[num][1].append(microscore)
            
            dic_report[num].append(metrics.classification_report(lab_encoder.inverse_transform(y_test), 
                                                                 lab_encoder.inverse_transform(preds), 
                                                                 output_dict=True))

            if confusion:
                cnf_matrix = confusion_matrix(lab_encoder.inverse_transform(y_test), 
                                              lab_encoder.inverse_transform(preds),
                                              labels=lab_encoder.classes_)
                np.set_printoptions(precision=2)
                plt.figure(fig_i)
                fig_i += 1
                plot_confusion_matrix(cnf_matrix, normalize=True, classes=lab_encoder.classes_,
                                      title=('Confusion matrix %s' % name(model)))
        
    return dico_score, models, dic_report
  
def macro(results):
    return np.mean(results[0]), np.std(results[0])

def micro(results):
    return np.mean(results[1]), np.std(results[1])

def describe_cv_result(results):
    for model in results.keys():
        print(model, 'macro f1', macro(results[model]))
        print(model, 'micro f1', micro(results[model]))

def describe_reports(reports):
    for model in reports.keys():
        frames = []
    for result in reports[model]:
        frames.append(pd.DataFrame.from_dict(result, orient='columns'))
    conc = pd.concat(frames)
    print(conc.groupby(by=conc.index, axis=0).mean())

In [ ]:
results, classifiers, reports = train_stratified([catboost], X, y, nsplits=5, confusion=True)
describe_cv_result(results)

In [ ]:
results, classifiers, reports = train_stratified([logreg], X, y, nsplits=5, confusion=True)
describe_cv_result(results)

In [65]:
describe_reports(reports)

           attribution_r  background_r     cause_r  comparison_m  condition_r  \
f1-score        0.815748      0.186804    0.560527      0.346042     0.698736   
precision       0.792869      0.222629    0.606078      0.478959     0.672605   
recall          0.841026      0.161111    0.522088      0.276667     0.727273   
support       117.000000     72.000000  249.000000     60.000000   165.000000   

           contrast_m  evaluation_r  evidence_r  preparation_r   purpose_r  \
f1-score     0.733007      0.429089    0.341182       0.666279    0.765918   
precision    0.769061      0.369879    0.351258       0.636403    0.776771   
recall       0.700503      0.511268    0.332500       0.700000    0.756667   
support    199.000000    142.000000   80.000000     162.000000  120.000000   

           sequence_m  accuracy    macro avg  weighted avg  
f1-score     0.662993  0.607919     0.564211      0.604476  
precision    0.632799  0.607919     0.573574      0.609308  
recall       0.69677

## Save the cool one 

In [ ]:
import glob
import os

files = sorted(glob.glob('./rst_pairs/*.edus'), key=lambda s: int(os.path.basename(s)[5]))
test = files[::5]
train = [file for file in files if not file in test]

In [ ]:
random_state = 41

In [ ]:
import pandas as pd

train_samples = []

for file in train:
    train_samples.append(pd.read_pickle(file.replace('.edus', '.gold.features')))

train_samples = pd.concat(train_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
#train_samples.relation = train_samples.relation.fillna(True)
#train_samples['genre'] = train_samples.filename.map(lambda row: row.split('_')[0])

In [ ]:
TARGET = 'category_id'

train_samples[TARGET] = train_samples[TARGET].replace(['cause-effect_r', 'effect_r'], 'cause_r')
train_samples[TARGET] = train_samples[TARGET].replace(['interpretation-evaluation_r', 'conclusion_r'], 'evaluation_r')

y_stat = train_samples[TARGET].value_counts()
drop_ys = y_stat[y_stat < 100].index

for dy in drop_ys:
    train_samples = train_samples[train_samples[TARGET] != dy]

In [ ]:
train_samples[TARGET].value_counts()

In [ ]:
test_samples = []

for file in test:
    test_samples.append(pd.read_pickle(file.replace('.edus', '.gold.features')))

test_samples = pd.concat(test_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
#test_samples.relation = test_samples.relation.fillna(True)
#test_samples['genre'] = test_samples.filename.map(lambda row: row.split('_')[0])

In [ ]:
TARGET = 'category_id'

test_samples[TARGET] = test_samples[TARGET].replace(['cause-effect_r', 'effect_r'], 'cause_r')
test_samples[TARGET] = test_samples[TARGET].replace(['interpretation-evaluation_r', 'conclusion_r'], 'evaluation_r')

#y_stat = test_samples[TARGET].value_counts()
#drop_ys = y_stat[y_stat < 100].index

for dy in drop_ys:
    test_samples = test_samples[test_samples[TARGET] != dy]

In [ ]:
TARGET = 'category_id'
y_train, X_train = train_samples[TARGET].to_frame(), train_samples.drop(TARGET, axis=1).drop(columns=['snippet_x', 'snippet_y', 'snippet_x_tmp', 'snippet_y_tmp', 'postags_x', 'postags_y', 'filename', 'order'])

In [ ]:
TARGET = 'category_id'
y_test, X_test = test_samples[TARGET].to_frame(), test_samples.drop(TARGET, axis=1).drop(columns=['snippet_x', 'snippet_y', 'snippet_x_tmp', 'snippet_y_tmp', 'postags_x', 'postags_y', 'filename', 'order'])

In [ ]:
categorical_feature_mask = X_train.dtypes==object
categorical_cols = X_train.columns[categorical_feature_mask].tolist()

In [ ]:
categorical_feature_mask = X_test.dtypes==object
categorical_cols = X_test.columns[categorical_feature_mask].tolist()

In [ ]:
categorical_cols

In [ ]:
X_train = X_train.drop(columns=categorical_cols)
X_test = X_test.drop(columns=categorical_cols)

In [ ]:
scaler = StandardScaler().fit(X_train)

X_scaled_np = scaler.transform(X_train)
X_train = pd.DataFrame(X_scaled_np, index=X_train.index)#, columns=X.columns)

In [ ]:
X_scaled_np = scaler.transform(X_test)
X_test = pd.DataFrame(X_scaled_np, index=X_test.index)#, columns=X.columns)

In [ ]:
pickle.dump(scaler, open('models/label_predictor/scaler.pkl', 'wb'))

In [ ]:
fs_catboost_plus_logreg.fit(X_train, y_train.values.ravel())

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.category_id.unique()

In [ ]:
y_test.category_id.unique()

In [ ]:
pickle.dump(fs_catboost_plus_logreg, open('models/label_predictor/model.pkl', 'wb'))

In [ ]:
from sklearn import metrics

predicted = fs_catboost_plus_logreg.predict(X_test)  
print('weighted f1: ', metrics.f1_score(y_test, predicted, average='weighted'))
print('macro f1: ', metrics.f1_score(y_test, predicted, average='macro'))
print()
print(metrics.classification_report(y_test, predicted))

### 4. Important features exploration

In [ ]:
fi = np.array(classifier.feature_importances_)
sorted_idx = np.argsort(fi)
print(np.count_nonzero(fi))

pd.set_option('display.max_rows', 150)
dd = pd.DataFrame({'Feature': np.array(X.keys())[sorted_idx], 'Importance': fi[sorted_idx][::-1]})
dd = dd[dd['Importance'] > 0]

In [ ]:
lexical = pd.concat([dd[dd.Feature.str.contains('tf_idf')], 
                     dd[dd.Feature.str.contains('upper_')],
                     dd[dd.Feature.str.contains('jac')],
                     dd[dd.Feature.str.contains('len')]]).drop_duplicates()
lexical = list(lexical.Feature)
print(len(lexical), lexical)

In [ ]:
morphosynt = pd.concat([dd[dd.Feature.str.islower() == False],
                        dd[dd.Feature.str.contains('morph')]]).drop_duplicates()
morphosynt = morphosynt.replace(['tf_idf_', 'upper', 'len_', 'stopwords'], np.NaN, regex=True).dropna()
morphosynt = morphosynt[morphosynt.Feature.str[-2:] != '_X']
morphosynt = morphosynt[morphosynt.Feature.str[-2:] != '_Y']

morphosynt = list(morphosynt.Feature)
print(len(morphosynt), morphosynt)

In [ ]:
textual = pd.concat([dd[dd.Feature.str[-2:] == '_X'],
                     dd[dd.Feature.str[-2:] == '_Y'],
                     dd[dd.Feature.str.contains('stopwords')]])
textual2 = pd.concat([dd[dd.Feature.str.contains('first')],
                      dd[dd.Feature.str.contains('last')]])
textual2 = textual2[textual2.Feature.str.islower()]
textual = pd.concat([textual, textual2])
textual = textual.replace(['tf_idf_', 'upper', 'len_', 'fPOS'], np.NaN, regex=True).dropna()

textual = list(textual.Feature)
print(len(textual), textual)

In [ ]:
textX = textual[textual.Feature.str[-2:] == '_x']
textY = textual[textual.Feature.str[-2:] == '_y']
firstPX = textX[textX.Feature.str.contains('first_pair')]
firstPY = textY[textY.Feature.str.contains('first_pair')]
lastPX = textX[textX.Feature.str.contains('last_pair')]
lastPY = textY[textY.Feature.str.contains('last_pair')]
countPX = textual[textual.Feature.str.contains('_X')]
countPY = textual[textual.Feature.str.contains('_Y')]

len(firstPX), len(firstPY), len(lastPX), len(lastPY), len(countPX), len(countPY)

In [ ]:
for _ in [lexical, morphosynt, textual]:
    print(len(_) / dd.shape[0] * 100)

In [42]:
model = CatBoostClassifier(
    iterations=1800,
    learning_rate=0.2,
    custom_loss=['F1'],
    random_seed=random_state,
    verbose=2,
    loss_function='MultiClass',
    task_type='GPU'
)

In [43]:
result = {
    feature: 0 for feature in X.keys()
}

In [44]:
#result = pd.DataFrame({'feature': X.keys(), 'score': np.zeros(X.shape[1])})

kf = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train, test in kf.split(X, y):
    X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]
    model.fit(X_train, y_train, eval_set=(X_test, y_test))
    fi = np.array(model.feature_importances_)
    sorted_idx = np.argsort(fi)
    dd = pd.DataFrame({'Feature': np.array(X.keys())[sorted_idx], 'Importance': fi[sorted_idx][::-1]})
    dd = dd[dd['Importance'] > 0].Feature.values
    for feat in dd:
        result[feat] += 1

0:	learn: 2.1554721	test: 2.1667875	best: 2.1667875 (0)	total: 66.3ms	remaining: 1m 59s
2:	learn: 1.9522507	test: 1.9666341	best: 1.9666341 (2)	total: 176ms	remaining: 1m 45s
4:	learn: 1.8514307	test: 1.8763824	best: 1.8763824 (4)	total: 299ms	remaining: 1m 47s
6:	learn: 1.7626402	test: 1.7995298	best: 1.7995298 (6)	total: 413ms	remaining: 1m 45s
8:	learn: 1.6992159	test: 1.7436214	best: 1.7436214 (8)	total: 498ms	remaining: 1m 39s
10:	learn: 1.6520811	test: 1.7046320	best: 1.7046320 (10)	total: 600ms	remaining: 1m 37s
12:	learn: 1.6147443	test: 1.6708535	best: 1.6708535 (12)	total: 679ms	remaining: 1m 33s
14:	learn: 1.5799209	test: 1.6406342	best: 1.6406342 (14)	total: 776ms	remaining: 1m 32s
16:	learn: 1.5418297	test: 1.6147333	best: 1.6147333 (16)	total: 915ms	remaining: 1m 35s
18:	learn: 1.5136280	test: 1.5997835	best: 1.5997835 (18)	total: 1.03s	remaining: 1m 36s
20:	learn: 1.4826297	test: 1.5767351	best: 1.5767351 (20)	total: 1.16s	remaining: 1m 37s
22:	learn: 1.4611168	test: 1.5

188:	learn: 0.7651760	test: 1.2844947	best: 1.2844947 (188)	total: 8.74s	remaining: 1m 14s
190:	learn: 0.7622606	test: 1.2842132	best: 1.2840483 (189)	total: 8.81s	remaining: 1m 14s
192:	learn: 0.7606405	test: 1.2840814	best: 1.2840483 (189)	total: 8.87s	remaining: 1m 13s
194:	learn: 0.7582882	test: 1.2830724	best: 1.2830724 (194)	total: 8.94s	remaining: 1m 13s
196:	learn: 0.7565900	test: 1.2829485	best: 1.2827440 (195)	total: 9s	remaining: 1m 13s
198:	learn: 0.7512178	test: 1.2824487	best: 1.2824487 (198)	total: 9.08s	remaining: 1m 13s
200:	learn: 0.7480724	test: 1.2822308	best: 1.2818717 (199)	total: 9.16s	remaining: 1m 12s
202:	learn: 0.7437557	test: 1.2808780	best: 1.2808780 (202)	total: 9.25s	remaining: 1m 12s
204:	learn: 0.7418829	test: 1.2807285	best: 1.2807285 (204)	total: 9.31s	remaining: 1m 12s
206:	learn: 0.7383059	test: 1.2801810	best: 1.2801810 (206)	total: 9.4s	remaining: 1m 12s
208:	learn: 0.7359561	test: 1.2795870	best: 1.2795870 (208)	total: 9.47s	remaining: 1m 12s
210

372:	learn: 0.5266893	test: 1.2473108	best: 1.2471616 (371)	total: 15.9s	remaining: 1m
374:	learn: 0.5254643	test: 1.2465104	best: 1.2465104 (374)	total: 15.9s	remaining: 1m
376:	learn: 0.5238356	test: 1.2467005	best: 1.2462107 (375)	total: 16s	remaining: 1m
378:	learn: 0.5230638	test: 1.2466937	best: 1.2462107 (375)	total: 16.1s	remaining: 1m
380:	learn: 0.5206417	test: 1.2470500	best: 1.2462107 (375)	total: 16.2s	remaining: 1m
382:	learn: 0.5182198	test: 1.2473550	best: 1.2462107 (375)	total: 16.2s	remaining: 1m
384:	learn: 0.5156454	test: 1.2470651	best: 1.2462107 (375)	total: 16.3s	remaining: 1m
386:	learn: 0.5140270	test: 1.2471794	best: 1.2462107 (375)	total: 16.4s	remaining: 59.9s
388:	learn: 0.5113085	test: 1.2482359	best: 1.2462107 (375)	total: 16.5s	remaining: 59.8s
390:	learn: 0.5100353	test: 1.2476410	best: 1.2462107 (375)	total: 16.6s	remaining: 59.7s
392:	learn: 0.5066615	test: 1.2472128	best: 1.2462107 (375)	total: 16.7s	remaining: 59.7s
394:	learn: 0.5052791	test: 1.247

558:	learn: 0.3891412	test: 1.2285592	best: 1.2279190 (554)	total: 22.9s	remaining: 50.9s
560:	learn: 0.3871910	test: 1.2288552	best: 1.2279190 (554)	total: 23s	remaining: 50.8s
562:	learn: 0.3864068	test: 1.2285982	best: 1.2279190 (554)	total: 23.1s	remaining: 50.7s
564:	learn: 0.3860104	test: 1.2281640	best: 1.2279190 (554)	total: 23.1s	remaining: 50.6s
566:	learn: 0.3841819	test: 1.2276650	best: 1.2276650 (566)	total: 23.2s	remaining: 50.5s
568:	learn: 0.3828510	test: 1.2281887	best: 1.2276650 (566)	total: 23.3s	remaining: 50.5s
570:	learn: 0.3811948	test: 1.2280910	best: 1.2276650 (566)	total: 23.4s	remaining: 50.4s
572:	learn: 0.3802273	test: 1.2282348	best: 1.2276650 (566)	total: 23.5s	remaining: 50.3s
574:	learn: 0.3788003	test: 1.2285883	best: 1.2276650 (566)	total: 23.6s	remaining: 50.2s
576:	learn: 0.3783669	test: 1.2281066	best: 1.2276650 (566)	total: 23.6s	remaining: 50.1s
578:	learn: 0.3767483	test: 1.2278356	best: 1.2276068 (577)	total: 23.7s	remaining: 50s
580:	learn: 0.

744:	learn: 0.2929896	test: 1.2157379	best: 1.2157024 (743)	total: 29.9s	remaining: 42.3s
746:	learn: 0.2919340	test: 1.2157590	best: 1.2154913 (745)	total: 29.9s	remaining: 42.2s
748:	learn: 0.2911145	test: 1.2159688	best: 1.2154913 (745)	total: 30s	remaining: 42.1s
750:	learn: 0.2905828	test: 1.2154365	best: 1.2154365 (750)	total: 30.1s	remaining: 42s
752:	learn: 0.2898059	test: 1.2153241	best: 1.2153241 (752)	total: 30.2s	remaining: 41.9s
754:	learn: 0.2892754	test: 1.2151719	best: 1.2151719 (754)	total: 30.2s	remaining: 41.8s
756:	learn: 0.2884858	test: 1.2157773	best: 1.2151719 (754)	total: 30.3s	remaining: 41.8s
758:	learn: 0.2881970	test: 1.2157802	best: 1.2151719 (754)	total: 30.4s	remaining: 41.7s
760:	learn: 0.2877755	test: 1.2158765	best: 1.2151719 (754)	total: 30.4s	remaining: 41.6s
762:	learn: 0.2867349	test: 1.2153084	best: 1.2151719 (754)	total: 30.5s	remaining: 41.5s
764:	learn: 0.2859218	test: 1.2159506	best: 1.2151719 (754)	total: 30.6s	remaining: 41.4s
766:	learn: 0.

...

894:	learn: 0.2289357	test: 1.2027021	best: 1.2023357 (891)	total: 35.7s	remaining: 36.1s
896:	learn: 0.2281881	test: 1.2028296	best: 1.2023357 (891)	total: 35.8s	remaining: 36s
898:	learn: 0.2275735	test: 1.2023403	best: 1.2023357 (891)	total: 35.8s	remaining: 35.9s
900:	learn: 0.2272748	test: 1.2028226	best: 1.2023357 (891)	total: 35.9s	remaining: 35.8s
902:	learn: 0.2263811	test: 1.2029766	best: 1.2023357 (891)	total: 36s	remaining: 35.8s
904:	learn: 0.2255444	test: 1.2032854	best: 1.2023357 (891)	total: 36.1s	remaining: 35.7s
906:	learn: 0.2250705	test: 1.2029365	best: 1.2023357 (891)	total: 36.1s	remaining: 35.6s
908:	learn: 0.2244549	test: 1.2030236	best: 1.2023357 (891)	total: 36.2s	remaining: 35.5s
910:	learn: 0.2241631	test: 1.2030319	best: 1.2023357 (891)	total: 36.3s	remaining: 35.4s
912:	learn: 0.2234397	test: 1.2026946	best: 1.2023357 (891)	total: 36.4s	remaining: 35.3s
914:	learn: 0.2230652	test: 1.2027438	best: 1.2023357 (891)	total: 36.4s	remaining: 35.2s
916:	learn: 0.

1080:	learn: 0.1752461	test: 1.1980638	best: 1.1976066 (1073)	total: 43s	remaining: 28.6s
1082:	learn: 0.1748140	test: 1.1980043	best: 1.1976066 (1073)	total: 43.1s	remaining: 28.5s
1084:	learn: 0.1744822	test: 1.1983835	best: 1.1976066 (1073)	total: 43.2s	remaining: 28.5s
1086:	learn: 0.1742210	test: 1.1983117	best: 1.1976066 (1073)	total: 43.3s	remaining: 28.4s
1088:	learn: 0.1740422	test: 1.1980226	best: 1.1976066 (1073)	total: 43.3s	remaining: 28.3s
1090:	learn: 0.1734051	test: 1.1978689	best: 1.1976066 (1073)	total: 43.4s	remaining: 28.2s
1092:	learn: 0.1729513	test: 1.1977324	best: 1.1976066 (1073)	total: 43.5s	remaining: 28.1s
1094:	learn: 0.1724513	test: 1.1972457	best: 1.1971688 (1093)	total: 43.6s	remaining: 28.1s
1096:	learn: 0.1720272	test: 1.1974886	best: 1.1971688 (1093)	total: 43.6s	remaining: 28s
1098:	learn: 0.1714414	test: 1.1973727	best: 1.1971688 (1093)	total: 43.7s	remaining: 27.9s
1100:	learn: 0.1711332	test: 1.1975958	best: 1.1971688 (1093)	total: 43.8s	remaining

1262:	learn: 0.1378733	test: 1.1990272	best: 1.1965776 (1106)	total: 50s	remaining: 21.3s
1264:	learn: 0.1373625	test: 1.1986892	best: 1.1965776 (1106)	total: 50.1s	remaining: 21.2s
1266:	learn: 0.1373087	test: 1.1987572	best: 1.1965776 (1106)	total: 50.1s	remaining: 21.1s
1268:	learn: 0.1372161	test: 1.1986438	best: 1.1965776 (1106)	total: 50.2s	remaining: 21s
1270:	learn: 0.1367212	test: 1.1983494	best: 1.1965776 (1106)	total: 50.3s	remaining: 20.9s
1272:	learn: 0.1364130	test: 1.1984755	best: 1.1965776 (1106)	total: 50.3s	remaining: 20.8s
1274:	learn: 0.1361899	test: 1.1983518	best: 1.1965776 (1106)	total: 50.4s	remaining: 20.8s
1276:	learn: 0.1360223	test: 1.1983771	best: 1.1965776 (1106)	total: 50.5s	remaining: 20.7s
1278:	learn: 0.1357347	test: 1.1986899	best: 1.1965776 (1106)	total: 50.6s	remaining: 20.6s
1280:	learn: 0.1353135	test: 1.1981386	best: 1.1965776 (1106)	total: 50.7s	remaining: 20.5s
1282:	learn: 0.1349512	test: 1.1982935	best: 1.1965776 (1106)	total: 50.7s	remaining

1442:	learn: 0.1110377	test: 1.2034460	best: 1.1965776 (1106)	total: 56.9s	remaining: 14.1s
1444:	learn: 0.1108382	test: 1.2033579	best: 1.1965776 (1106)	total: 57s	remaining: 14s
1446:	learn: 0.1106789	test: 1.2033238	best: 1.1965776 (1106)	total: 57.1s	remaining: 13.9s
1448:	learn: 0.1103345	test: 1.2039376	best: 1.1965776 (1106)	total: 57.2s	remaining: 13.8s
1450:	learn: 0.1100427	test: 1.2044415	best: 1.1965776 (1106)	total: 57.2s	remaining: 13.8s
1452:	learn: 0.1098807	test: 1.2045533	best: 1.1965776 (1106)	total: 57.3s	remaining: 13.7s
1454:	learn: 0.1096617	test: 1.2046893	best: 1.1965776 (1106)	total: 57.4s	remaining: 13.6s
1456:	learn: 0.1094525	test: 1.2044763	best: 1.1965776 (1106)	total: 57.4s	remaining: 13.5s
1458:	learn: 0.1091620	test: 1.2044698	best: 1.1965776 (1106)	total: 57.5s	remaining: 13.4s
1460:	learn: 0.1088538	test: 1.2039179	best: 1.1965776 (1106)	total: 57.6s	remaining: 13.4s
1462:	learn: 0.1087314	test: 1.2037268	best: 1.1965776 (1106)	total: 57.7s	remaining

1624:	learn: 0.0901602	test: 1.2082312	best: 1.1965776 (1106)	total: 1m 3s	remaining: 6.89s
1626:	learn: 0.0898900	test: 1.2084427	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.81s
1628:	learn: 0.0897560	test: 1.2087559	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.73s
1630:	learn: 0.0895729	test: 1.2089858	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.65s
1632:	learn: 0.0894171	test: 1.2090622	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.57s
1634:	learn: 0.0892176	test: 1.2093341	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.49s
1636:	learn: 0.0890883	test: 1.2094613	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.41s
1638:	learn: 0.0889833	test: 1.2097301	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.33s
1640:	learn: 0.0887721	test: 1.2099670	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.26s
1642:	learn: 0.0885231	test: 1.2096283	best: 1.1965776 (1106)	total: 1m 4s	remaining: 6.18s
1644:	learn: 0.0883853	test: 1.2095078	best: 1.1965776 (1106)	total: 1m 4s	remai

In [49]:
rr = pd.DataFrame.from_dict(result, orient='index')
rr[rr[0] > 0].index.values

array(['token_begin_y', 'token_end_y', 'len_w_x', ..., 'eucl_embed_dist',
       'manh_embed_dist', 'ang_cos_sim'], dtype=object)

In [50]:
rr[rr[0] > 0]

,0
token_begin_y,2
token_end_y,8
len_w_x,2
sentence_begin_x,1
sentence_begin_y,5
sentence_end_y,8
number_sents_x,10
number_sents_y,6
same_sentence,4
root_in_y,9


In [51]:
list(rr[rr[0] == 0].index.values)

['token_begin_x',
 'len_w_y',
 'common_root_position',
 'len_av_x',
 'len_av_y',
 'st_up_x',
 'st_up_y',
 'du_st_up_y',
 'VerbForm_Fin_x',
 'fPOS_CONJ_x',
 'fPOS_NOUN_y',
 'first_pair_\\d+_x',
 'first_pair_для\\b_x',
 'last_pair_\\._x',
 'last_pair_,_x',
 'first_ADV_X_y',
 'first_pair_если_y',
 'first_pair_«_y',
 'first_pair_\\bа\\b_y',
 'first_pair_\\bчто\\b_y',
 'first_pair_чтобы_y',
 'first_pair_однако_y',
 'first_pair_\\bно\\b_y',
 'first_pair_поэтому_y',
 'first_pair_позволя_y',
 'first_pair_пос[кт]ольку_y',
 'first_pair_для_y',
 'morph_canberra',
 '\\bчто\\b_count_y',
 '\\bи\\b_count_y',
 '\\bно\\b_count_y',
 '\\bта_count_y',
 'tf_idf_x_1',
 'tf_idf_x_3',
 'tf_idf_x_4',
 'tf_idf_x_7',
 'tf_idf_x_12',
 'tf_idf_x_30',
 'tf_idf_x_49',
 'tf_idf_x_54',
 'tf_idf_x_56',
 'tf_idf_x_62',
 'tf_idf_x_63',
 'tf_idf_x_79',
 'tf_idf_x_108',
 'tf_idf_x_162',
 'tf_idf_x_168',
 'tf_idf_x_207',
 'tf_idf_x_225',
 'tf_idf_x_263',
 'tf_idf_x_281',
 'tf_idf_x_285',
 'tf_idf_y_1',
 'tf_idf_y_8',
 'tf_i

### 5. Feature selection

In [16]:
! pip install feature_selector

     |████████████████████████████████| 184kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 12.2MB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 716kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 15.0MB 1.5MB/s eta 0:00:011
     |████████████████████████████████| 11.8MB 2.5MB/s eta 0:00:01
  Created wheel for seaborn: filename=seaborn-0.8.1-cp36-none-any.whl size=184834 sha256=7485fc9ec0d6492671e22b12e7146441b58de5bd45e8c787954a09e48df36a7c
  Stored in directory: /root/.cache/pip/wheels/26/0a/44/53ddd89769e62f7c6691976375b86c6492e7dd20a2d3970e32
Successfully built seaborn
ERROR: lime 0.1.1.32 has requirement matplotlib==2, but you'll have matplotlib 2.1.2 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
ERROR: catboost 0.20.1 has requirement numpy>=1.16.0, but you'll have numpy 1.

In [37]:
from feature_selector import FeatureSelector

fs = FeatureSelector(data=X, labels=y)

In [38]:
fs.identify_missing(missing_threshold=0.1)

0 features with greater than 0.10 missing values.



####  - Find features with single unique value

In [39]:
fs.identify_single_unique()

0 features with a single unique value.



In [40]:
# list of single unique values features
fs.ops['single_unique']

[]

#### - Find collinear features

In [ ]:
fs = FeatureSelector(data=X, labels=y)
fs.identify_collinear(correlation_threshold=.99, one_hot=False)

In [ ]:
fs.plot_collinear()

In [ ]:
# list of collinear features to remove
fs.ops['collinear']

### 6. Symmetry regressor implementation

In [ ]:
import pandas as pd

df = pd.read_pickle(PATH+DATAFILE)
TARGET = 'category_id'

def get_classes(df, y_name, classes):
    res = []
    for class_name in classes:
        res.append(df[df[y_name] == class_name])

    return pd.concat(res).sample(frac=1).reset_index(drop=True)

for_bin_regr = [
    'joint_m',
    'elaboration_r',
    'restatement_m',
    'concession_r',
    'solutionhood_r',
    'antithesis_r',
]

df_bin = get_classes(df, 'category_id', for_bin_regr)

In [ ]:
TARGET = 'category_id'

names = list(df[TARGET].unique())

replace_map = {
    TARGET: {
        k: v for k, v in [(name, name[-1]) for name in names]
    }
}

df_bin = df_bin.replace(replace_map).replace({TARGET: {'r': 0, 'm': 1}})

In [ ]:
df_bin.category_id.value_counts()

In [ ]:
TARGET = 'category_id'

y_bin, X_bin = df_bin[TARGET].to_frame(), df_bin.drop(TARGET, axis=1).drop(columns=['snippet_x', 'snippet_y', 'snippet_x_tmp', 'snippet_y_tmp', 'order', 'filename'])

In [ ]:
lgbm_param_bin =  {
    'tree_learner': 'feature',
    'task': 'train',
    'random_state': random_state,
    'metric': 'binary_logloss',
    'feature_fraction': 0.8,
    'colsample_bytree' : 0.8,
    'boosting_type': 'dart',
    'application': 'regression',
    'num_iterations': 150
}

lgb_bin_class = lgb.LGBMClassifier(**lgbm_param_bin)

X_train, X_test, y_train, y_test = train_test_split(X_bin, y_bin, shuffle=True, test_size=.1, random_state=random_state)

feature_selector = SelectFromModel(LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1'))
model_single = Pipeline([('feature_selector', feature_selector), 
                         ('classifier', lgb_bin_class)])

model = BaggingClassifier(base_estimator=model_single, 
                          n_estimators=7,
                          max_samples=1.0, 
                          max_features=0.6, 
                          bootstrap=True, 
                          random_state=random_state)
model.fit(X_train, y_train)

predicted = model.predict(X_test)
print(metrics.classification_report(y_test, predicted))
print(confusion_matrix(y_test, predicted))

In [ ]:
filename = 'symmetry_predictor.save'
pickle.dump(model, open(PATH + filename, 'wb'))

### 7. Classifiers parameters tuning

#### - GridSearch

In [ ]:
param_grid = {'C': [0.0001, 0.005, 0.001] }
clf = GridSearchCV(LinearSVC(random_state=random_state), param_grid, cv=5, verbose=2)
clf.fit(X.values, y.values.ravel())

In [ ]:
clf.best_params_

#### - RandomSearch 

In [ ]:
lgb_model = lgb.LGBMClassifier()

lgbm_param =  {
    'task': ['train'],
    'boosting_type': ['gbdt'],
    'objective': ['multiclass'],
    'num_class': [6,],
    'metric': ['multi_error'],
    "learning_rate": list(np.arange(0.05, 1, 0.25)),
    "num_leaves": list(range(32, 128, 16)),
    "max_depth": list(range(2, 17, 2)),
    "reg_alpha": [.15],
    "reg_lambda": [.15],
}

clf = RandomSearchCV(lgb_model, lgbm_param, n_jobs=8, cv=5)
clf.fit(scaling.transform(X.values), y.values.ravel())

print("Best parameters:")
print(clf.best_params_)
print()
print("Grid score:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

## Nuclearity types classification

1. Data cleaning: obtain the data and clean it (X, y)
 - transformations: normalization, scaling
2. Classifiers & CV-training

#### 1. Data cleaning 

In [ ]:
df = pd.read_pickle(PATH+DATAFILE)
TARGET = 'order'
df[TARGET].value_counts()

In [ ]:
unique_type_list = df[TARGET].unique()
replace_map = {TARGET : {k: v for k,v in zip(unique_type_list, list(range(0,len(unique_type_list))))}}
y, X = df[TARGET].to_frame(), df.drop(TARGET, axis=1).drop(columns=['snippet_x', 'snippet_y', 'snippet_x_tmp', 'snippet_y_tmp', 'filename', 'category_id'])

In [ ]:
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X)
scl = StandardScaler().fit(X.values)

X_scaled_np = scl.transform(X.values)
X_scaled_df = pd.DataFrame(X_scaled_np, index=X.index, columns=X.columns)

X = X_scaled_df

#### 2. Classifiers & CV-training

In [ ]:
lsvc = LinearSVC(random_state=random_state, C=0.001)
logreg = LogisticRegression(random_state=random_state,
                            solver='lbfgs',
                            n_jobs=8,
                            C=0.001,
                            multi_class='auto')

lgbm_param =  {
    'tree_learner': 'feature',
    'task': 'train',
    'scale_pos_weight': 155,
    'reg_lambda': 0.11,
    'reg_alpha': 0.11,
    'random_state': random_state,
    'num_leaves': 36,
    'num_class': 3,
    'min_split_gain': 0.76,
    'min_child_weight': 0.11,
    'metric': 'multi_logloss',
    'learning_rate': 0.1,
    'feature_fraction': 0.45,
    'boosting_type': 'dart',
    'bagging_fraction': 0.9,
    'application': 'multiclass',
    'num_iterations': 500,
}
lgb_class = lgb.LGBMClassifier(**lgbm_param)
     
lgbm_plus_bagger_lsvc = VotingClassifier([('LGBM', lgb_class), ('bagger', bagger_lsvc)], voting='soft')

def construct_simple_nn_classifier(input_length, output_length):
    input_features = Input(shape=(input_length,))
    l = Dense(100, kernel_initializer='he_uniform')(input_features)
    l = Activation('relu')(l)
    l = BatchNormalization(axis = 1)(l)
    l = Dropout(0.5)(l)
    outputs = Dense(output_length, activation='softmax')(l)

    model = SklearnWrapperKerasEstimator(inputs=[input_features], outputs=outputs)
    adam = Adam(lr=0.001, amsgrad=True)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['mean_squared_error', 'acc'])

    return model
  
nnclassifier = construct_simple_nn_classifier(input_length=X.shape[1], output_length=len(np.unique(y)))

catboost = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.1,
    custom_loss=['F1'],
    random_seed=random_state,
    verbose=0,
    loss_function='MultiClass',
    task_type='GPU',
)

fs_catboost = Pipeline([
  ('feature_selection', SelectFromModel(LogisticRegression(penalty='l1', C=1.))),
  ('classification', catboost)
])

fs_lgbm = Pipeline([
  ('feature_selection', SelectFromModel(LogisticRegression(penalty='l1', C=1.))),
  ('classification', lgb_class)
])

nnclassifier_smote = Pipeline([('smt', smt), ('nnclassifier', nnclassifier)])

name = lambda x : str(x).split('(')[0]

In [ ]:
results, classifiers, reports = train_stratified([nnclassifier_smote, 
                                                  logreg, 
                                                  lsvc, 
                                                  catboost, 
                                                  lgb_class, 
                                                  fs_catboost, 
                                                  fs_lgbm], X, y, nsplits=5, confusion=False)

describe_cv_result(results)

### Genre classification and feature selection 

In [62]:
df.head()

,snippet_x,category_id,snippet_y,order,filename,token_begin_x,token_begin_y,token_end_y,len_w_x,len_w_y,...,cos_embed_dist,eucl_embed_dist,manh_embed_dist,ang_cos_sim,snippet_x_tmp,snippet_y_tmp,postags_x,postags_y,postags_chrf,inverted_text_length
0,Конструктивная схема отражает элементный соста...,joint_m,Потоковая схема представляет модель преобразов...,SN,comp_5,0.041009,0.051262,0.061514,13,13,...,0.109308,0.305743,4.203936,0.069727,конструктивный_ADJ схема_NOUN отражать_VERB эл...,потоковый_ADJ схема_NOUN представлять_VERB мод...,ADJ NOUN VERB ADJ NOUN NOUN CONJ ADJ NOUN X VE...,ADJ NOUN VERB NOUN NOUN ADJ CONJ ADJ NOUN ADJ ...,0.708096,0.000789
1,Для случая прозрачности жидкости и дна ротора ...,contrast_m,в противном случае - зеркалом.,SN,comp_5,0.874606,0.883281,0.888013,11,6,...,0.419396,0.570621,7.969773,0.275516,для_ADP случай_NOUN прозрачность_NOUN жидкость...,в_ADP противный_ADJ случай_NOUN -_ зеркало_NOU...,ADP NOUN NOUN NOUN CONJ NOUN NOUN NOUN VERB NO...,ADP ADJ NOUN X NOUN X,0.128997,0.000789
2,Чтобы при вращении менялась не только форма то...,purpose_r,"в стенках вырезаем отверстия, закрытые эластич...",SN,comp_5,0.888013,0.899054,0.906151,14,9,...,0.294811,0.492028,6.802300,0.190513,чтобы_CONJ при_ADP вращение_NOUN меняться_VERB...,в_ADP стенка_NOUN вырезать_VERB отверстие_NOUN...,CONJ ADP NOUN VERB PART PART NOUN NOUN X CONJ ...,ADP NOUN VERB NOUN X VERB ADJ NOUN X,0.288543,0.000789
3,Чтобы при вращении менялась не только форма то...,same-unit_m,Тогда,SN,comp_5,0.888013,0.906151,0.906940,23,1,...,0.500000,0.591375,7.906408,0.333333,чтобы_CONJ при_ADP вращение_NOUN меняться_VERB...,тогда_ADV,CONJ ADP NOUN VERB PART PART NOUN NOUN X CONJ ...,ADV,0.002915,0.000789
4,при изменении скорости вращения,condition_r,будет меняться также и уровень жидкости.,SN,comp_5,0.906940,0.910095,0.915615,4,7,...,0.354322,0.607601,8.464622,0.230577,при_ADP изменение_NOUN скорость_NOUN вращение_...,быть_VERB меняться_VERB также_ADV и_CONJ урове...,ADP NOUN NOUN NOUN,VERB VERB ADV CONJ NOUN NOUN X,0.393506,0.000789


In [63]:
df.filename = df.filename.map(lambda row: row.split('_')[0])

In [64]:
y, X = df['filename'].to_frame(), df.drop(
    columns=[TARGET, 'snippet_x', 'snippet_y', 'snippet_x_tmp', 'snippet_y_tmp', 
             'filename', 'order', 'postags_x', 'postags_y'])

In [ ]:
catboost = CatBoostClassifier(
    iterations=1800,
    learning_rate=0.2,
    custom_loss=['F1'],
    random_seed=random_state,
    verbose=2,
    loss_function='MultiClass',
    task_type='GPU',
    plot=True
)

X_train, X_test, y_train, y_test = train_test_split(X, y)
catboost.fit(X_train, y_train, eval_set=(X_test, y_test))

In [77]:
print(metrics.classification_report(catboost.predict(X_test), y_test))

              precision    recall  f1-score   support

        comp       0.97      0.98      0.98      1290
        ling       0.98      0.98      0.98      1523
        news       1.00      0.98      0.99      1246

    accuracy                           0.98      4059
   macro avg       0.98      0.98      0.98      4059
weighted avg       0.98      0.98      0.98      4059



In [79]:
fi = np.array(catboost.feature_importances_)
sorted_idx = np.argsort(fi)
print(np.count_nonzero(fi))

pd.set_option('display.max_rows', 150)
dd = pd.DataFrame({'Feature': np.array(X.keys())[sorted_idx], 'Importance': fi[sorted_idx][::-1]})
dd = dd[dd['Importance'] > 0]

1525


In [80]:
dd

,Feature,Importance
0,last_pair_сказа_y,3.524660e+01
1,first_NOUN_NOUN_y,1.068461e+00
2,last_NOUN_NOUN_y,9.697332e-01
3,\bявля[ею]_count_x,8.994186e-01
4,last_NOUN_PART_y,8.974654e-01
5,first_NOUN_VERB_y,8.415694e-01
6,\bвопрос_count_x,7.753412e-01
7,morph_matching,7.351682e-01
8,last_NUM_NOUN_y,6.955793e-01
9,first_PART_VERB_y,6.485521e-01
